In [48]:
import pandas as pd
import numpy as np
import glob
import warnings
warnings.simplefilter("ignore", category=Warning)


In [49]:
def avg_last_execution(files_path, drift_detector, generator, metric = 'detected changes'):
    
    results = []
    for file in files_path:
        #print(file)
        df = pd.read_csv(file)
        #print(df.iloc[-1][metric])
        value = df.iloc[-1][metric]
        if value != "?":
            results.append(round(float(value), 2))
        
    
    tuple_results = tuple([np.mean(results), np.std(results), drift_detector, generator])
   
    #print(f'avg of all last execution: {np.mean(median)}')
    return tuple_results 

In [50]:
drift_detectors_params = [
                            {"id": "DDM", "drift_name": "DDM", "params": ""},    \
                            {"id": "EDDM", "drift_name": "EDDM", "params": ""},   \
                            {"id": "ADWIN", "drift_name": "ADWINChangeDetector", "params": ""},  \
                            {"id": "ECDD", "drift_name": "EWMAChartDM", "params": ""},   \
                            {"id": "STEPD", "drift_name": "STEPD", "params": ""},  \
                            {"id": "SeqDrift2", "drift_name": "SeqDrift2ChangeDetector", "params": ""},  \
                            {"id": "SEED", "drift_name": "SEEDChangeDetector", "params": ""},   \
                            {"id": "HDDM_A_Test", "drift_name": "HDDM_A_Test", "params": ""},  \
                            {"id": "HDDM_W_Test", "drift_name": "HDDM_W_Test", "params": ""},  \
                            #{"id": "FHDDM", "drift_name": "FHDDM", "params": ""},  \
                            #{"id": "FTDD", "drift_name": "FTDD", "params": ""},   \
                            {"id": "RDDM_30", "drift_name": "RDDM", "params": "-n 30 -w 2 -o 3"}, \
                            {"id": "RDDM","drift_name": "RDDM", "params": ""},   \
                            #{"id": "WSTD","drift_name": "WSTD", "params": ""},    \
                            {"id": "HypothesisTestDetectorKs","drift_name": "HypothesisTestDetector", "params": "-h ks"},    \
                            {"id": "HypothesisTestDetectorWrs","drift_name": "HypothesisTestDetector", "params": "-h wrs"},    \
                            {"id": "HypothesisTestDetectorTt","drift_name": "HypothesisTestDetector", "params": "-h tt"},    \
                            {"id": "MultivariateDetector","drift_name": "moa.learners.ChangeDetectorEnsembleMultivariateLearner", "params": "-d (PageHinkleyDM -d 0.002) -p 20"},    \
                            {"id": "Studd","drift_name": "moa.learners.StuddLearner", "params": "-b (meta.AdaptiveRandomForest -u  -q -w) -s (meta.AdaptiveRandomForest -u  -q -w) -d (PageHinkleyDM -d 0.002) -w 500"},    \
                            {"id": "NoChangeDetector","drift_name": "NoChangeDetectorNaive", "params": "-n 1000"}                        
                        ]

# Abrupt HT 100K - Percentage no labels: 95% - Metric: Detected Changes

In [51]:
classifier = "trees.HoeffdingTree"
drift_size = 100000
#partialy_drift options: 95, 96, 97, 98, 99
partialy_drift = 95
type_drift = "Abrupt"
#metric name options: detected changes, detected warnings, delay detection (average), delay true detection (average), MTFA (average), MDR, true changes detected, 
metric_name = "detected changes"
dict_drift_type = {"abrupt_agraw1": "AGRAW1", "abrupt_agraw2": "AGRAW2", "abrupt_led": "LED", "abrupt_mixed": "MIXED", "abrupt_randomRBF": "RANDOM_RBF",  
                    "abrupt_sine": "SINE", "abrupt_waveform": "WAVEFORM"}

df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        ##trees.HoeffdingTree_ADWIN_100000_95_0_AGRAW1_Abrupt
        files_path = glob.glob(f'../results/{drift_key}/{classifier}_{drift["id"]}_{drift_size}_{partialy_drift}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value, metric = metric_name)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,HypothesisTestDetectorKs,HypothesisTestDetectorTt,HypothesisTestDetectorWrs,MultivariateDetector,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2,Studd
generator,,,,,,,,,,,,,,,,,
AGRAW1,64.133333,54.533333,183.866667,300.700000,49.800000,80.333333,356.700000,310.700000,1699.833333,61.433333,89.766667,85.066667,64.733333,81.433333,81.066667,42.733333,0.0
AGRAW2,94.533333,155.700000,243.833333,213.766667,85.766667,89.333333,193.466667,195.333333,3048.000000,61.433333,89.766667,86.200000,75.966667,163.866667,211.300000,149.000000,0.1
LED,0.000000,0.000000,68.233333,3.933333,3.466667,24.133333,338.033333,305.033333,2432.766667,0.000000,89.766667,0.000000,20.233333,396.133333,147.533333,246.366667,5.4
MIXED,91.233333,82.733333,169.533333,149.433333,71.933333,89.366667,165.433333,160.533333,2746.400000,10.266667,89.766667,86.100000,68.833333,216.633333,87.433333,170.200000,0.0
RANDOM_RBF,152.800000,75.700000,104.766667,109.866667,78.866667,82.500000,219.000000,264.366667,2662.733333,0.000000,89.766667,91.466667,70.433333,581.300000,302.733333,286.300000,19.8
SINE,80.866667,109.033333,209.133333,223.000000,84.233333,82.666667,185.000000,210.300000,2825.200000,0.333333,89.766667,98.233333,78.633333,177.766667,125.700000,172.133333,0.0
WAVEFORM,0.000000,2.000000,147.166667,68.133333,1.100000,3.100000,296.200000,276.000000,2693.633333,0.000000,89.766667,4.166667,0.700000,3.866667,26.033333,0.700000,0.0


# Abrupt HT 100K - Percentage no labels: 96% - Metric: Detected Changes

In [52]:
classifier = "trees.HoeffdingTree"
drift_size = 100000
#partialy_drift options: 95, 96, 97, 98, 99
partialy_drift = 96
type_drift = "Abrupt"
#metric name options: detected changes, detected warnings, delay detection (average), delay true detection (average), MTFA (average), MDR, true changes detected, 
metric_name = "detected changes"
dict_drift_type = {"abrupt_agraw1": "AGRAW1", "abrupt_agraw2": "AGRAW2", "abrupt_led": "LED", "abrupt_mixed": "MIXED", "abrupt_randomRBF": "RANDOM_RBF",  
                    "abrupt_sine": "SINE", "abrupt_waveform": "WAVEFORM"}

df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        ##trees.HoeffdingTree_ADWIN_100000_95_0_AGRAW1_Abrupt
        files_path = glob.glob(f'../results/{drift_key}/{classifier}_{drift["id"]}_{drift_size}_{partialy_drift}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value, metric = metric_name)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,HypothesisTestDetectorKs,HypothesisTestDetectorTt,HypothesisTestDetectorWrs,MultivariateDetector,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2,Studd
generator,,,,,,,,,,,,,,,,,
AGRAW1,62.966667,49.800000,179.766667,302.700000,54.266667,94.066667,387.633333,320.666667,1773.766667,63.900000,90.0,80.333333,37.133333,82.900000,87.666667,67.366667,0.0
AGRAW2,99.133333,196.700000,254.366667,209.033333,99.366667,109.533333,224.466667,225.866667,3038.066667,61.866667,90.0,109.266667,108.966667,193.233333,281.000000,192.066667,0.0
LED,0.000000,0.000000,68.100000,3.433333,0.933333,15.633333,313.666667,317.666667,2346.100000,0.000000,90.0,0.000000,24.900000,441.033333,165.366667,260.800000,5.5
MIXED,114.933333,102.166667,176.733333,195.133333,100.566667,112.100000,208.333333,170.633333,2728.433333,12.466667,90.0,107.900000,106.733333,227.000000,118.433333,211.400000,0.0
RANDOM_RBF,179.400000,103.266667,133.633333,136.466667,98.500000,104.366667,275.833333,308.300000,2719.533333,0.000000,90.0,103.200000,101.133333,578.400000,390.266667,274.433333,33.3
SINE,121.500000,127.966667,252.533333,264.200000,112.266667,95.566667,216.733333,235.333333,2860.066667,1.066667,90.0,145.766667,112.100000,231.800000,171.433333,204.600000,0.0
WAVEFORM,0.000000,0.133333,147.566667,106.900000,1.833333,3.266667,274.033333,300.133333,2719.900000,0.000000,90.0,9.466667,0.100000,1.400000,24.400000,0.366667,0.0


# Abrupt HT 100K - Percentage no labels: 97% - Metric: Detected Changes

In [53]:
classifier = "trees.HoeffdingTree"
drift_size = 100000
#partialy_drift options: 95, 96, 97, 98, 99
partialy_drift = 97
type_drift = "Abrupt"
#metric name options: detected changes, detected warnings, delay detection (average), delay true detection (average), MTFA (average), MDR, true changes detected, 
metric_name = "detected changes"
dict_drift_type = {"abrupt_agraw1": "AGRAW1", "abrupt_agraw2": "AGRAW2", "abrupt_led": "LED", "abrupt_mixed": "MIXED", "abrupt_randomRBF": "RANDOM_RBF",  
                    "abrupt_sine": "SINE", "abrupt_waveform": "WAVEFORM"}

df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        ##trees.HoeffdingTree_ADWIN_100000_95_0_AGRAW1_Abrupt
        files_path = glob.glob(f'../results/{drift_key}/{classifier}_{drift["id"]}_{drift_size}_{partialy_drift}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value, metric = metric_name)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,HypothesisTestDetectorKs,HypothesisTestDetectorTt,HypothesisTestDetectorWrs,MultivariateDetector,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2,Studd
generator,,,,,,,,,,,,,,,,,
AGRAW1,61.400000,59.100000,167.166667,378.133333,64.900000,107.400000,392.300000,331.866667,1747.166667,52.266667,99.233333,107.700000,51.400000,93.366667,94.500000,71.900000,0.000000
AGRAW2,77.966667,138.800000,284.700000,255.800000,127.133333,132.466667,229.633333,233.033333,3006.033333,52.266667,99.233333,154.033333,110.066667,230.500000,287.833333,194.733333,0.000000
LED,0.000000,0.000000,41.666667,0.033333,4.966667,12.466667,323.033333,307.533333,2341.833333,0.000000,99.233333,0.000000,35.566667,418.533333,203.666667,198.333333,2.000000
MIXED,131.866667,146.333333,216.566667,213.800000,128.300000,143.466667,193.366667,212.566667,2692.266667,3.333333,99.233333,160.500000,151.100000,314.966667,163.066667,259.000000,0.000000
RANDOM_RBF,213.766667,132.800000,177.700000,170.166667,134.166667,165.000000,350.066667,361.800000,2757.566667,0.000000,99.233333,137.333333,148.866667,530.500000,484.066667,266.733333,30.166667
SINE,129.566667,172.900000,265.633333,311.966667,134.500000,129.866667,194.933333,256.366667,2851.866667,1.200000,99.233333,158.333333,139.966667,267.233333,179.700000,240.066667,0.000000
WAVEFORM,0.000000,0.000000,165.266667,97.500000,1.633333,0.966667,237.033333,312.333333,2723.533333,0.000000,99.233333,6.666667,0.000000,10.866667,22.133333,4.466667,0.000000


# Abrupt HT 100K - Percentage no labels: 98% - Metric: Detected Changes

In [54]:
classifier = "trees.HoeffdingTree"
drift_size = 100000
#partialy_drift options: 95, 96, 97, 98, 99
partialy_drift = 98
type_drift = "Abrupt"
#metric name options: detected changes, detected warnings, delay detection (average), delay true detection (average), MTFA (average), MDR, true changes detected, 
metric_name = "detected changes"
dict_drift_type = {"abrupt_agraw1": "AGRAW1", "abrupt_agraw2": "AGRAW2", "abrupt_led": "LED", "abrupt_mixed": "MIXED", "abrupt_randomRBF": "RANDOM_RBF",  
                    "abrupt_sine": "SINE", "abrupt_waveform": "WAVEFORM"}

df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        ##trees.HoeffdingTree_ADWIN_100000_95_0_AGRAW1_Abrupt
        files_path = glob.glob(f'../results/{drift_key}/{classifier}_{drift["id"]}_{drift_size}_{partialy_drift}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value, metric = metric_name)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,HypothesisTestDetectorKs,HypothesisTestDetectorTt,HypothesisTestDetectorWrs,MultivariateDetector,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2,Studd
generator,,,,,,,,,,,,,,,,,
AGRAW1,68.733333,78.933333,225.400000,443.766667,114.566667,168.333333,400.800000,413.800000,1681.633333,106.800000,58.166667,107.166667,67.366667,98.300000,119.600000,81.533333,0.000000
AGRAW2,14.900000,187.000000,311.166667,350.900000,160.733333,172.866667,292.900000,247.366667,2854.800000,106.800000,58.166667,236.600000,124.366667,198.233333,430.700000,180.466667,0.000000
LED,0.000000,0.000000,49.400000,11.266667,0.966667,16.433333,355.700000,384.100000,2408.433333,0.000000,58.166667,0.000000,47.466667,425.400000,303.000000,235.966667,0.000000
MIXED,231.600000,206.933333,278.266667,267.966667,202.566667,209.533333,243.800000,269.566667,2719.366667,0.133333,58.166667,208.466667,179.433333,396.233333,243.000000,241.600000,0.000000
RANDOM_RBF,286.733333,226.033333,208.933333,242.733333,230.266667,198.933333,368.433333,371.766667,2622.433333,0.000000,58.166667,185.466667,193.700000,545.366667,641.966667,197.100000,20.633333
SINE,214.900000,198.333333,336.700000,309.366667,199.200000,225.200000,233.533333,257.766667,2839.033333,0.000000,58.166667,223.200000,177.566667,354.933333,205.666667,245.100000,0.000000
WAVEFORM,0.000000,0.000000,146.433333,107.300000,0.000000,11.533333,227.700000,294.433333,2764.600000,0.000000,58.166667,6.900000,0.000000,14.800000,27.200000,10.566667,0.000000


# Abrupt HT 100K - Percentage no labels: 99% - Metric: Detected Changes

In [55]:
classifier = "trees.HoeffdingTree"
drift_size = 100000
#partialy_drift options: 95, 96, 97, 98, 99
partialy_drift = 99
type_drift = "Abrupt"
#metric name options: detected changes, detected warnings, delay detection (average), delay true detection (average), MTFA (average), MDR, true changes detected, 
metric_name = "detected changes"
dict_drift_type = {"abrupt_agraw1": "AGRAW1", "abrupt_agraw2": "AGRAW2", "abrupt_led": "LED", "abrupt_mixed": "MIXED", "abrupt_randomRBF": "RANDOM_RBF",  
                    "abrupt_sine": "SINE", "abrupt_waveform": "WAVEFORM"}

df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        ##trees.HoeffdingTree_ADWIN_100000_95_0_AGRAW1_Abrupt
        files_path = glob.glob(f'../results/{drift_key}/{classifier}_{drift["id"]}_{drift_size}_{partialy_drift}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value, metric = metric_name)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,HypothesisTestDetectorKs,HypothesisTestDetectorTt,HypothesisTestDetectorWrs,MultivariateDetector,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2,Studd
generator,,,,,,,,,,,,,,,,,
AGRAW1,54.900000,174.433333,231.333333,738.933333,111.733333,252.266667,296.466667,345.533333,1764.900000,297.7,58.733333,171.666667,112.366667,120.133333,153.566667,128.900000,0.0
AGRAW2,8.266667,110.933333,407.100000,433.333333,110.500000,193.300000,382.666667,396.266667,3103.100000,293.2,58.733333,163.200000,65.266667,199.500000,333.600000,118.066667,0.0
LED,0.000000,0.000000,31.100000,21.866667,0.000000,4.600000,341.966667,433.533333,2225.166667,0.0,58.733333,0.000000,117.666667,417.400000,278.433333,300.133333,0.0
MIXED,383.166667,386.533333,460.933333,454.000000,379.533333,425.133333,472.933333,482.333333,2486.933333,0.0,58.733333,408.966667,408.666667,662.633333,422.966667,319.166667,0.0
RANDOM_RBF,124.033333,385.700000,423.500000,513.533333,437.033333,368.700000,408.633333,525.800000,2531.900000,0.0,58.733333,386.233333,403.366667,424.700000,751.866667,201.666667,16.4
SINE,291.733333,315.200000,482.966667,457.266667,307.300000,356.333333,369.133333,384.766667,2699.533333,0.0,58.733333,413.566667,332.900000,595.400000,361.400000,278.600000,0.0
WAVEFORM,0.000000,0.000000,118.166667,194.833333,0.000000,3.666667,402.200000,363.566667,2627.600000,0.0,58.733333,16.400000,0.000000,3.233333,33.800000,7.366667,0.0


# Gradual HT 100K - Percentage no labels: 95% - Metric: Detected Changes

In [56]:
classifier = "trees.HoeffdingTree"
drift_size = 100000
#partialy_drift options: 95, 96, 97, 98, 99
partialy_drift = 95
type_drift = "Gradual"
#metric name options: detected changes, detected warnings, delay detection (average), delay true detection (average), MTFA (average), MDR, true changes detected, 
metric_name = "detected changes"
dict_drift_type = {"gradual_agraw1": "AGRAW1", "gradual_agraw2": "AGRAW2", "gradual_led": "LED", 
                    "gradual_mixed": "MIXED", "gradual_randomRBF": "RANDOM_RBF", "gradual_sine": "SINE", "gradual_waveform": "WAVEFORM"}

df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        ##trees.HoeffdingTree_ADWIN_100000_95_0_AGRAW1_Abrupt
        files_path = glob.glob(f'../results/{drift_key}/{classifier}_{drift["id"]}_{drift_size}_{partialy_drift}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value, metric = metric_name)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,HypothesisTestDetectorKs,HypothesisTestDetectorTt,HypothesisTestDetectorWrs,MultivariateDetector,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2,Studd
generator,,,,,,,,,,,,,,,,,
AGRAW1,65.033333,54.600000,183.833333,312.933333,53.666667,81.800000,356.566667,291.733333,1753.233333,56.6,89.766667,75.333333,43.433333,71.400000,77.500000,44.966667,0.000000
AGRAW2,99.266667,182.500000,241.566667,209.466667,74.200000,96.333333,198.366667,199.833333,3051.600000,56.6,89.766667,71.266667,73.366667,158.866667,208.266667,157.666667,0.000000
LED,0.000000,0.000000,66.866667,3.933333,0.666667,18.866667,356.900000,308.333333,2372.666667,0.0,89.766667,0.000000,0.000000,418.833333,187.233333,227.500000,0.133333
MIXED,99.100000,91.066667,174.700000,131.566667,86.433333,86.400000,115.133333,136.933333,2662.000000,7.6,89.766667,98.233333,89.133333,221.133333,103.900000,175.133333,0.000000
RANDOM_RBF,156.166667,82.000000,102.933333,112.966667,73.666667,76.166667,242.700000,238.366667,2660.066667,0.0,89.766667,82.033333,77.433333,573.433333,308.066667,265.500000,30.633333
SINE,81.800000,112.333333,211.266667,234.433333,79.066667,79.466667,162.166667,180.566667,2875.366667,0.0,89.766667,105.866667,94.533333,182.366667,118.933333,167.266667,0.000000
WAVEFORM,0.000000,2.233333,166.966667,100.566667,1.266667,3.633333,260.500000,291.133333,2694.966667,0.0,89.766667,7.600000,3.066667,4.433333,27.000000,0.700000,0.200000


# Gradual HT 100K - Percentage no labels: 96% - Metric: Detected Changes

In [57]:
classifier = "trees.HoeffdingTree"
drift_size = 100000
#partialy_drift options: 95, 96, 97, 98, 99
partialy_drift = 96
type_drift = "Gradual"
#metric name options: detected changes, detected warnings, delay detection (average), delay true detection (average), MTFA (average), MDR, true changes detected, 
metric_name = "detected changes"
dict_drift_type = {"gradual_agraw1": "AGRAW1", "gradual_agraw2": "AGRAW2", "gradual_led": "LED", 
                    "gradual_mixed": "MIXED", "gradual_randomRBF": "RANDOM_RBF", "gradual_sine": "SINE", "gradual_waveform": "WAVEFORM"}

df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        ##trees.HoeffdingTree_ADWIN_100000_95_0_AGRAW1_Abrupt
        files_path = glob.glob(f'../results/{drift_key}/{classifier}_{drift["id"]}_{drift_size}_{partialy_drift}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value, metric = metric_name)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,HypothesisTestDetectorKs,HypothesisTestDetectorTt,HypothesisTestDetectorWrs,MultivariateDetector,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2,Studd
generator,,,,,,,,,,,,,,,,,
AGRAW1,52.766667,55.033333,176.833333,343.900000,61.733333,115.100000,345.533333,285.100000,1806.000000,77.800000,90.0,69.500000,53.033333,79.333333,84.300000,73.700000,0.000000
AGRAW2,113.033333,200.400000,252.433333,196.366667,98.833333,109.600000,205.233333,197.666667,3026.400000,76.400000,90.0,121.433333,100.000000,204.933333,269.800000,187.166667,0.000000
LED,0.000000,0.000000,60.500000,3.433333,0.933333,15.833333,335.166667,335.433333,2280.900000,0.000000,90.0,0.000000,0.000000,417.366667,208.433333,261.066667,0.000000
MIXED,113.400000,117.433333,186.566667,172.066667,96.733333,93.800000,127.633333,123.233333,2678.266667,10.833333,90.0,98.633333,96.233333,264.033333,119.133333,214.033333,0.000000
RANDOM_RBF,230.200000,88.933333,118.400000,135.800000,110.000000,101.200000,307.133333,289.866667,2695.233333,0.000000,90.0,96.200000,101.766667,526.133333,363.666667,284.833333,26.966667
SINE,120.733333,122.566667,218.566667,248.200000,101.033333,108.266667,193.166667,226.200000,2866.900000,0.000000,90.0,132.300000,108.900000,206.066667,154.533333,207.000000,0.000000
WAVEFORM,0.000000,0.300000,164.666667,112.400000,0.466667,7.833333,267.933333,302.900000,2702.066667,0.000000,90.0,13.366667,3.866667,3.300000,25.666667,0.366667,0.000000


# Gradual HT 100K - Percentage no labels: 97% - Metric: Detected Changes

In [58]:
classifier = "trees.HoeffdingTree"
drift_size = 100000
#partialy_drift options: 95, 96, 97, 98, 99
partialy_drift = 97
type_drift = "Gradual"
#metric name options: detected changes, detected warnings, delay detection (average), delay true detection (average), MTFA (average), MDR, true changes detected, 
metric_name = "detected changes"
dict_drift_type = {"gradual_agraw1": "AGRAW1", "gradual_agraw2": "AGRAW2", "gradual_led": "LED", 
                    "gradual_mixed": "MIXED", "gradual_randomRBF": "RANDOM_RBF", "gradual_sine": "SINE", "gradual_waveform": "WAVEFORM"}

df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        ##trees.HoeffdingTree_ADWIN_100000_95_0_AGRAW1_Abrupt
        files_path = glob.glob(f'../results/{drift_key}/{classifier}_{drift["id"]}_{drift_size}_{partialy_drift}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value, metric = metric_name)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,HypothesisTestDetectorKs,HypothesisTestDetectorTt,HypothesisTestDetectorWrs,MultivariateDetector,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2,Studd
generator,,,,,,,,,,,,,,,,,
AGRAW1,63.066667,65.800000,243.766667,360.400000,55.033333,116.100000,365.066667,317.633333,1775.933333,61.700000,99.233333,115.433333,58.900000,59.700000,84.666667,68.633333,0.0
AGRAW2,124.666667,245.433333,257.166667,261.400000,133.233333,146.433333,228.133333,217.800000,2937.733333,58.166667,99.233333,150.966667,128.300000,232.733333,340.100000,179.600000,0.0
LED,0.000000,0.000000,49.000000,0.033333,0.000000,16.800000,337.800000,311.900000,2430.100000,0.000000,99.233333,0.000000,0.000000,445.966667,179.400000,192.700000,0.0
MIXED,127.300000,135.466667,210.233333,185.666667,118.900000,128.433333,168.266667,174.933333,2604.666667,6.366667,99.233333,148.100000,131.766667,309.166667,141.333333,263.766667,0.0
RANDOM_RBF,226.733333,135.300000,152.900000,167.900000,133.633333,138.766667,308.400000,300.566667,2635.700000,0.000000,99.233333,134.866667,129.133333,559.200000,469.433333,262.433333,24.4
SINE,122.466667,159.433333,217.300000,271.233333,130.833333,135.233333,180.200000,219.966667,2778.233333,0.000000,99.233333,153.300000,140.233333,277.833333,149.366667,236.866667,0.0
WAVEFORM,0.000000,0.866667,156.466667,113.933333,0.000000,0.900000,281.633333,254.433333,2784.900000,0.000000,99.233333,12.833333,0.833333,5.733333,27.166667,4.233333,0.0


# Gradual HT 100K - Percentage no labels: 98% - Metric: Detected Changes

In [59]:
classifier = "trees.HoeffdingTree"
drift_size = 100000
#partialy_drift options: 95, 96, 97, 98, 99
partialy_drift = 98
type_drift = "Gradual"
#metric name options: detected changes, detected warnings, delay detection (average), delay true detection (average), MTFA (average), MDR, true changes detected, 
metric_name = "detected changes"
dict_drift_type = {"gradual_agraw1": "AGRAW1", "gradual_agraw2": "AGRAW2", "gradual_led": "LED", 
                    "gradual_mixed": "MIXED", "gradual_randomRBF": "RANDOM_RBF", "gradual_sine": "SINE", "gradual_waveform": "WAVEFORM"}

df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        ##trees.HoeffdingTree_ADWIN_100000_95_0_AGRAW1_Abrupt
        files_path = glob.glob(f'../results/{drift_key}/{classifier}_{drift["id"]}_{drift_size}_{partialy_drift}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value, metric = metric_name)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,HypothesisTestDetectorKs,HypothesisTestDetectorTt,HypothesisTestDetectorWrs,MultivariateDetector,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2,Studd
generator,,,,,,,,,,,,,,,,,
AGRAW1,108.466667,89.366667,213.566667,386.700000,61.333333,161.033333,432.633333,328.733333,1756.400000,94.200000,58.166667,124.433333,75.433333,76.300000,78.466667,90.733333,0.0
AGRAW2,38.000000,255.133333,316.833333,333.433333,197.000000,177.766667,265.433333,271.200000,3052.800000,82.033333,58.166667,210.666667,134.800000,213.400000,431.033333,138.433333,0.0
LED,0.000000,0.000000,40.766667,11.266667,2.333333,21.733333,287.966667,371.233333,2400.066667,0.000000,58.166667,0.000000,0.000000,361.466667,301.833333,199.200000,0.0
MIXED,215.166667,199.266667,251.333333,283.666667,222.000000,211.900000,196.033333,225.066667,2554.266667,2.733333,58.166667,211.066667,200.833333,429.966667,207.966667,245.066667,0.0
RANDOM_RBF,294.066667,198.633333,214.900000,220.033333,202.533333,212.433333,346.900000,365.566667,2541.966667,0.000000,58.166667,222.966667,190.166667,514.266667,491.233333,186.533333,41.9
SINE,218.133333,233.366667,315.100000,307.200000,191.566667,191.866667,276.533333,270.166667,2749.233333,0.000000,58.166667,204.300000,207.466667,380.366667,222.966667,258.366667,0.0
WAVEFORM,0.000000,0.333333,177.833333,114.800000,0.000000,1.333333,263.666667,268.433333,2681.933333,0.000000,58.166667,15.766667,0.333333,7.533333,13.000000,18.133333,0.0


# Gradual HT 100K - Percentage no labels: 99% - Metric: Detected Changes

In [60]:
classifier = "trees.HoeffdingTree"
drift_size = 100000
#partialy_drift options: 95, 96, 97, 98, 99
partialy_drift = 99
type_drift = "Gradual"
#metric name options: detected changes, detected warnings, delay detection (average), delay true detection (average), MTFA (average), MDR, true changes detected, 
metric_name = "detected changes"
dict_drift_type = {"gradual_agraw1": "AGRAW1", "gradual_agraw2": "AGRAW2", "gradual_led": "LED", 
                    "gradual_mixed": "MIXED", "gradual_randomRBF": "RANDOM_RBF", "gradual_sine": "SINE", "gradual_waveform": "WAVEFORM"}

df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        ##trees.HoeffdingTree_ADWIN_100000_95_0_AGRAW1_Abrupt
        files_path = glob.glob(f'../results/{drift_key}/{classifier}_{drift["id"]}_{drift_size}_{partialy_drift}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value, metric = metric_name)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,HypothesisTestDetectorKs,HypothesisTestDetectorTt,HypothesisTestDetectorWrs,MultivariateDetector,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2,Studd
generator,,,,,,,,,,,,,,,,,
AGRAW1,89.266667,147.600000,235.133333,643.500000,179.000000,195.266667,353.566667,322.666667,1675.300000,232.466667,58.733333,218.966667,133.933333,103.633333,155.566667,151.733333,0.000000
AGRAW2,5.300000,123.966667,328.633333,386.566667,115.100000,262.400000,349.800000,315.866667,2912.233333,229.600000,58.733333,192.100000,55.500000,174.200000,414.533333,108.766667,0.000000
LED,0.000000,0.000000,45.900000,21.866667,0.000000,17.900000,308.133333,402.600000,2218.366667,0.000000,58.733333,0.000000,0.000000,410.666667,401.800000,275.066667,0.000000
MIXED,378.066667,454.933333,449.666667,423.200000,378.733333,442.633333,465.033333,571.600000,2598.733333,0.000000,58.733333,408.066667,429.733333,739.133333,490.466667,330.066667,0.000000
RANDOM_RBF,114.833333,367.833333,390.100000,441.200000,370.066667,406.300000,467.200000,551.700000,2474.333333,0.000000,58.733333,394.233333,432.033333,409.266667,762.433333,235.666667,13.966667
SINE,257.100000,372.033333,451.500000,434.200000,323.733333,384.966667,334.300000,475.033333,2831.033333,0.000000,58.733333,410.033333,328.133333,589.300000,367.166667,315.600000,0.000000
WAVEFORM,0.000000,3.700000,137.433333,212.033333,0.000000,0.000000,350.566667,398.166667,2727.300000,0.000000,58.733333,13.533333,3.700000,7.766667,42.000000,0.000000,0.000000


# Abrupt HT 100K - Percentage no labels: 95% - Metric: delay true detection (average)

In [61]:
classifier = "trees.HoeffdingTree"
drift_size = 100000
#partialy_drift options: 95, 96, 97, 98, 99
partialy_drift = 95
type_drift = "Abrupt"
#metric name options: detected changes, detected warnings, delay detection (average), delay true detection (average), MTFA (average), MDR, true changes detected, 
metric_name = "delay true detection (average)"
dict_drift_type = {"abrupt_agraw1": "AGRAW1", "abrupt_agraw2": "AGRAW2", "abrupt_led": "LED", "abrupt_mixed": "MIXED", "abrupt_randomRBF": "RANDOM_RBF",  
                    "abrupt_sine": "SINE", "abrupt_waveform": "WAVEFORM"}

df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        ##trees.HoeffdingTree_ADWIN_100000_95_0_AGRAW1_Abrupt
        files_path = glob.glob(f'../results/{drift_key}/{classifier}_{drift["id"]}_{drift_size}_{partialy_drift}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value, metric = metric_name)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,HypothesisTestDetectorKs,HypothesisTestDetectorTt,HypothesisTestDetectorWrs,MultivariateDetector,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2,Studd
generator,,,,,,,,,,,,,,,,,
AGRAW1,121.167333,254.479667,65.141333,43.330667,203.552000,104.734333,55.314667,61.065000,2.600333,49.939286,426.782,148.327000,248.701000,291.537000,175.794333,292.265667,NaN
AGRAW2,105.890690,44.057857,12.355333,38.762000,48.446667,36.273333,94.533333,66.299667,0.827000,49.939286,426.782,54.626000,121.446207,40.555333,14.377000,102.527667,3305.330000
LED,NaN,NaN,581.621667,0.000000,427.853333,1035.108333,74.025000,80.245333,1.030667,NaN,426.782,NaN,156.330333,37.086667,36.036667,48.917333,665.732222
MIXED,35.904000,29.248667,9.292333,14.944333,13.120000,12.894000,65.464000,53.142667,0.957000,2567.661579,426.782,17.034667,34.422333,12.295333,9.195667,85.101000,NaN
RANDOM_RBF,24.085333,41.526000,19.240333,39.078667,14.050333,15.777333,42.809333,38.439000,1.013333,NaN,426.782,23.463667,49.610333,17.225000,5.522333,62.446333,961.101333
SINE,56.169333,36.044667,32.967667,15.322000,12.516333,16.130333,42.490333,55.523667,1.062333,462.000000,426.782,15.196333,37.688667,17.370000,8.898667,84.950333,NaN
WAVEFORM,NaN,76.070000,202.185333,48.147391,24.240000,1622.478333,78.765333,94.181333,0.826333,NaN,426.782,281.760000,217.330000,0.000000,575.839091,0.000000,NaN


# Abrupt HT 100K - Percentage no labels: 96% - Metric: delay true detection (average)

In [62]:
classifier = "trees.HoeffdingTree"
drift_size = 100000
#partialy_drift options: 95, 96, 97, 98, 99
partialy_drift = 96
type_drift = "Abrupt"
#metric name options: detected changes, detected warnings, delay detection (average), delay true detection (average), MTFA (average), MDR, true changes detected, 
metric_name = "delay true detection (average)"
dict_drift_type = {"abrupt_agraw1": "AGRAW1", "abrupt_agraw2": "AGRAW2", "abrupt_led": "LED", "abrupt_mixed": "MIXED", "abrupt_randomRBF": "RANDOM_RBF",  
                    "abrupt_sine": "SINE", "abrupt_waveform": "WAVEFORM"}

df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        ##trees.HoeffdingTree_ADWIN_100000_95_0_AGRAW1_Abrupt
        files_path = glob.glob(f'../results/{drift_key}/{classifier}_{drift["id"]}_{drift_size}_{partialy_drift}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value, metric = metric_name)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,HypothesisTestDetectorKs,HypothesisTestDetectorTt,HypothesisTestDetectorWrs,MultivariateDetector,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2,Studd
generator,,,,,,,,,,,,,,,,,
AGRAW1,98.332667,221.567000,87.100000,27.924000,219.611333,127.816667,45.892333,68.659333,3.028667,101.575667,627.802,226.669000,297.029000,215.197667,128.596333,270.382667,NaN
AGRAW2,107.832000,49.642000,16.233000,45.231034,54.150667,38.975333,57.739667,67.434000,1.209000,103.578667,627.802,48.034333,117.606333,50.922333,17.370000,157.408333,NaN
LED,NaN,NaN,561.935667,0.000000,149.595000,1726.276667,60.675333,77.009333,1.318667,NaN,627.802,NaN,182.560000,52.216333,49.279655,61.984138,850.980
MIXED,42.201333,26.780333,4.020000,14.440000,10.503667,16.557333,20.279667,22.563000,1.290000,1267.722500,627.802,13.058000,22.614333,15.173333,8.951667,76.885333,NaN
RANDOM_RBF,28.590667,36.744000,9.244667,35.693000,16.611000,16.187333,55.001667,24.059333,1.031667,NaN,627.802,24.915000,38.803333,24.908000,8.305333,84.062667,574.146
SINE,42.953333,28.362667,11.902000,12.191667,13.559667,18.903000,50.407000,33.058000,1.275000,473.000000,627.802,11.970667,33.597333,17.463667,7.788333,84.572667,NaN
WAVEFORM,NaN,1200.750000,243.339667,23.174762,16.050000,924.410000,89.126667,105.632333,0.953333,NaN,627.802,153.074286,1601.000000,0.000000,440.863529,0.000000,NaN


# Abrupt HT 100K - Percentage no labels: 97% - Metric: delay true detection (average)

In [63]:
classifier = "trees.HoeffdingTree"
drift_size = 100000
#partialy_drift options: 95, 96, 97, 98, 99
partialy_drift = 97
type_drift = "Abrupt"
#metric name options: detected changes, detected warnings, delay detection (average), delay true detection (average), MTFA (average), MDR, true changes detected, 
metric_name = "delay true detection (average)"
dict_drift_type = {"abrupt_agraw1": "AGRAW1", "abrupt_agraw2": "AGRAW2", "abrupt_led": "LED", "abrupt_mixed": "MIXED", "abrupt_randomRBF": "RANDOM_RBF",  
                    "abrupt_sine": "SINE", "abrupt_waveform": "WAVEFORM"}

df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        ##trees.HoeffdingTree_ADWIN_100000_95_0_AGRAW1_Abrupt
        files_path = glob.glob(f'../results/{drift_key}/{classifier}_{drift["id"]}_{drift_size}_{partialy_drift}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value, metric = metric_name)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,HypothesisTestDetectorKs,HypothesisTestDetectorTt,HypothesisTestDetectorWrs,MultivariateDetector,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2,Studd
generator,,,,,,,,,,,,,,,,,
AGRAW1,371.136667,225.644667,88.228333,38.677000,109.267241,187.393333,67.969000,76.822000,4.622000,20.042222,562.037,168.834667,253.997333,213.789333,134.649333,267.589333,NaN
AGRAW2,166.772000,89.258333,16.649333,23.526207,46.315333,43.488333,75.331667,84.146333,1.340667,20.042222,562.037,43.961333,90.300833,60.165333,22.319667,163.624333,NaN
LED,NaN,NaN,827.579655,0.000000,963.733333,1570.968462,81.105000,101.491333,2.090333,NaN,562.037,NaN,137.351000,56.549667,68.860690,96.259259,171.290000
MIXED,43.659667,19.018333,2.423333,12.516333,11.083667,14.450667,51.326667,49.045000,1.656000,3883.280000,562.037,12.736000,20.765667,15.567000,10.171667,86.306333,NaN
RANDOM_RBF,38.635667,37.505667,4.846000,31.660000,12.293000,13.227333,38.241333,33.044333,1.638333,NaN,562.037,23.345333,34.612667,35.424000,8.844000,92.140667,2189.233182
SINE,59.646667,24.950667,7.102667,9.804333,15.557667,17.952667,47.732667,32.501333,1.757000,291.890000,562.037,12.797667,33.163000,19.891000,10.301667,91.258333,NaN
WAVEFORM,NaN,NaN,198.612000,65.438333,79.880000,256.345000,131.671333,110.918333,1.377000,NaN,562.037,156.936667,NaN,78.316667,136.249167,79.640000,NaN


# Abrupt HT 100K - Percentage no labels: 98% - Metric: delay true detection (average)

In [64]:
classifier = "trees.HoeffdingTree"
drift_size = 100000
#partialy_drift options: 95, 96, 97, 98, 99
partialy_drift = 98
type_drift = "Abrupt"
#metric name options: detected changes, detected warnings, delay detection (average), delay true detection (average), MTFA (average), MDR, true changes detected, 
metric_name = "delay true detection (average)"
dict_drift_type = {"abrupt_agraw1": "AGRAW1", "abrupt_agraw2": "AGRAW2", "abrupt_led": "LED", "abrupt_mixed": "MIXED", "abrupt_randomRBF": "RANDOM_RBF",  
                    "abrupt_sine": "SINE", "abrupt_waveform": "WAVEFORM"}

df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        ##trees.HoeffdingTree_ADWIN_100000_95_0_AGRAW1_Abrupt
        files_path = glob.glob(f'../results/{drift_key}/{classifier}_{drift["id"]}_{drift_size}_{partialy_drift}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value, metric = metric_name)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,HypothesisTestDetectorKs,HypothesisTestDetectorTt,HypothesisTestDetectorWrs,MultivariateDetector,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2,Studd
generator,,,,,,,,,,,,,,,,,
AGRAW1,258.207667,121.289667,119.626333,31.531000,75.707667,72.975333,84.785667,67.476000,6.814000,53.814286,689.073333,110.410000,180.557333,186.693667,111.852333,453.580000,NaN
AGRAW2,108.440000,49.380909,17.567333,23.488966,61.978571,55.242333,73.024667,75.383000,2.275000,53.814286,689.073333,35.736667,108.442727,169.338333,23.647333,215.364286,NaN
LED,NaN,NaN,1055.318800,0.000000,589.930000,1586.183000,72.255667,80.498667,3.131667,NaN,689.073333,NaN,223.309667,59.197667,41.281429,103.321667,NaN
MIXED,44.048667,24.772333,5.534000,13.851000,10.654333,14.904667,39.245667,38.158000,2.589333,3772.250000,689.073333,11.207333,22.540667,19.638667,10.242667,158.471333,NaN
RANDOM_RBF,46.358333,33.378333,6.305333,26.434333,12.843667,17.715000,46.004000,42.048000,2.576000,NaN,689.073333,24.034000,38.883667,43.372000,12.582000,193.168667,636.188333
SINE,58.232333,31.141333,3.969000,13.107333,14.767333,14.975000,59.685333,43.174333,3.107000,NaN,689.073333,14.374333,36.035667,22.057000,13.610000,157.790333,NaN
WAVEFORM,NaN,NaN,213.560000,46.790556,NaN,176.133333,140.716000,108.996000,1.892333,NaN,689.073333,694.381250,NaN,224.185000,386.491818,12.286667,NaN


# Abrupt HT 100K - Percentage no labels: 99% - Metric: delay true detection (average)

In [65]:
classifier = "trees.HoeffdingTree"
drift_size = 100000
#partialy_drift options: 95, 96, 97, 98, 99
partialy_drift = 99
type_drift = "Abrupt"
#metric name options: detected changes, detected warnings, delay detection (average), delay true detection (average), MTFA (average), MDR, true changes detected, 
metric_name = "delay true detection (average)"
dict_drift_type = {"abrupt_agraw1": "AGRAW1", "abrupt_agraw2": "AGRAW2", "abrupt_led": "LED", "abrupt_mixed": "MIXED", "abrupt_randomRBF": "RANDOM_RBF",  
                    "abrupt_sine": "SINE", "abrupt_waveform": "WAVEFORM"}

df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        ##trees.HoeffdingTree_ADWIN_100000_95_0_AGRAW1_Abrupt
        files_path = glob.glob(f'../results/{drift_key}/{classifier}_{drift["id"]}_{drift_size}_{partialy_drift}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value, metric = metric_name)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,HypothesisTestDetectorKs,HypothesisTestDetectorTt,HypothesisTestDetectorWrs,MultivariateDetector,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2,Studd
generator,,,,,,,,,,,,,,,,,
AGRAW1,499.468824,147.770345,91.883000,19.024667,98.804000,59.587667,66.249000,77.813000,8.445333,30.716923,801.488571,72.364000,184.317241,224.137333,91.426000,615.189667,NaN
AGRAW2,70.030000,31.743636,24.806000,49.801034,107.334000,105.758148,35.057931,70.332333,4.183667,30.378846,801.488571,57.551429,63.757273,230.028214,92.942759,799.769375,NaN
LED,NaN,NaN,329.235385,0.000000,NaN,249.766667,88.897241,46.411724,6.709333,NaN,801.488571,NaN,433.458667,90.683333,81.845185,151.206897,NaN
MIXED,63.354667,14.722000,4.504667,9.825667,12.934667,14.650333,23.099667,24.109000,6.417000,NaN,801.488571,10.800000,18.680333,24.330000,13.337667,153.900333,NaN
RANDOM_RBF,222.587931,37.096333,10.675667,23.291000,14.003333,24.249333,113.201000,29.632759,4.900000,NaN,801.488571,22.976667,53.709667,45.930333,31.660000,590.922333,123.925
SINE,80.375333,24.348333,21.999000,17.313000,16.673667,18.980000,36.693333,38.443448,5.183333,NaN,801.488571,17.829667,37.500333,34.897333,16.355000,169.693667,NaN
WAVEFORM,NaN,NaN,317.505909,60.931667,NaN,140.790000,54.680000,60.808621,4.231333,NaN,801.488571,109.092500,NaN,16.306667,176.345000,196.010000,NaN


# Gradual HT 100K - Percentage no labels: 95% - Metric: delay true detection (average)

In [66]:
classifier = "trees.HoeffdingTree"
drift_size = 100000
#partialy_drift options: 95, 96, 97, 98, 99
partialy_drift = 95
type_drift = "Gradual"
#metric name options: detected changes, detected warnings, delay detection (average), delay true detection (average), MTFA (average), MDR, true changes detected, 
metric_name = "delay true detection (average)"
dict_drift_type = {"gradual_agraw1": "AGRAW1", "gradual_agraw2": "AGRAW2", "gradual_led": "LED", 
                    "gradual_mixed": "MIXED", "gradual_randomRBF": "RANDOM_RBF", "gradual_sine": "SINE", "gradual_waveform": "WAVEFORM"}

df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        ##trees.HoeffdingTree_ADWIN_100000_95_0_AGRAW1_Abrupt
        files_path = glob.glob(f'../results/{drift_key}/{classifier}_{drift["id"]}_{drift_size}_{partialy_drift}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value, metric = metric_name)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,HypothesisTestDetectorKs,HypothesisTestDetectorTt,HypothesisTestDetectorWrs,MultivariateDetector,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2,Studd
generator,,,,,,,,,,,,,,,,,
AGRAW1,127.105667,355.661000,70.038667,47.124667,142.878667,99.348333,50.755667,68.260000,2.752000,60.689286,426.782,153.767333,367.660667,370.618667,318.643667,319.112000,NaN
AGRAW2,96.738621,45.364138,47.156333,41.656333,55.487667,33.601333,83.084333,89.216000,0.982667,60.689286,426.782,70.940667,166.553667,48.712667,41.801667,95.923000,NaN
LED,NaN,NaN,613.537333,0.000000,666.450000,634.863810,59.990000,86.516667,1.062000,NaN,426.782,NaN,NaN,38.446333,50.714000,44.198000,2091.750000
MIXED,34.735000,21.399333,143.694333,14.357667,12.142333,12.154000,212.172667,172.045333,1.245333,2729.280556,426.782,12.351333,23.589333,15.246667,59.136333,81.866333,NaN
RANDOM_RBF,25.237333,44.700000,77.514000,38.570333,18.694667,17.867667,87.568667,104.490333,0.947000,NaN,426.782,24.898333,45.147667,20.847667,4.414667,72.056667,657.122667
SINE,53.984000,25.754000,100.459667,11.100667,17.723667,16.994667,185.232333,128.661667,1.177000,NaN,426.782,21.365000,30.915333,15.601333,9.311667,88.160333,NaN
WAVEFORM,NaN,242.650000,175.067333,51.480833,1609.893333,5757.116000,122.729000,90.424667,0.774333,NaN,426.782,1726.583571,1027.154000,817.120000,887.198750,0.000000,153.330000


# Gradual HT 100K - Percentage no labels: 96% - Metric: delay true detection (average)

In [67]:
classifier = "trees.HoeffdingTree"
drift_size = 100000
#partialy_drift options: 95, 96, 97, 98, 99
partialy_drift = 96
type_drift = "Gradual"
#metric name options: detected changes, detected warnings, delay detection (average), delay true detection (average), MTFA (average), MDR, true changes detected, 
metric_name = "delay true detection (average)"
dict_drift_type = {"gradual_agraw1": "AGRAW1", "gradual_agraw2": "AGRAW2", "gradual_led": "LED", 
                    "gradual_mixed": "MIXED", "gradual_randomRBF": "RANDOM_RBF", "gradual_sine": "SINE", "gradual_waveform": "WAVEFORM"}

df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        ##trees.HoeffdingTree_ADWIN_100000_95_0_AGRAW1_Abrupt
        files_path = glob.glob(f'../results/{drift_key}/{classifier}_{drift["id"]}_{drift_size}_{partialy_drift}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value, metric = metric_name)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,HypothesisTestDetectorKs,HypothesisTestDetectorTt,HypothesisTestDetectorWrs,MultivariateDetector,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2,Studd
generator,,,,,,,,,,,,,,,,,
AGRAW1,197.486333,176.409000,65.596000,23.757333,165.867333,147.997667,72.964333,124.567333,3.135667,278.742759,627.802,230.379333,230.182667,234.486000,167.070667,245.307333,NaN
AGRAW2,94.146667,44.742000,52.649667,43.355172,61.840000,46.215333,79.346667,106.681000,1.158667,276.909655,627.802,47.018333,103.955333,63.282667,24.935667,122.912333,NaN
LED,NaN,NaN,707.826207,0.000000,568.410000,1470.171667,81.773667,83.802000,1.283667,NaN,627.802,NaN,NaN,49.757000,97.957241,76.536897,NaN
MIXED,45.117333,20.290667,160.817333,12.284333,13.005667,15.737000,176.070667,118.654667,1.613667,1047.583077,627.802,13.776667,24.446000,12.470333,5.865333,76.440667,NaN
RANDOM_RBF,21.389000,44.160333,25.634333,34.081667,14.378000,16.103000,58.599333,91.193333,1.414333,NaN,627.802,28.351667,42.929333,27.612333,4.696667,93.275667,668.434333
SINE,48.862333,30.417667,68.938333,15.244333,17.280000,17.087667,106.729667,79.728333,1.333667,NaN,627.802,12.484667,30.924333,17.636333,17.588000,84.947333,NaN
WAVEFORM,NaN,999.000000,189.335000,52.455833,1192.430000,4895.320000,106.037667,97.624333,0.966000,NaN,627.802,425.528571,787.038000,1051.267500,916.912778,0.000000,NaN


# Gradual HT 100K - Percentage no labels: 97% - Metric: delay true detection (average)

In [68]:
classifier = "trees.HoeffdingTree"
drift_size = 100000
#partialy_drift options: 95, 96, 97, 98, 99
partialy_drift = 97
type_drift = "Gradual"
#metric name options: detected changes, detected warnings, delay detection (average), delay true detection (average), MTFA (average), MDR, true changes detected, 
metric_name = "delay true detection (average)"
dict_drift_type = {"gradual_agraw1": "AGRAW1", "gradual_agraw2": "AGRAW2", "gradual_led": "LED", 
                    "gradual_mixed": "MIXED", "gradual_randomRBF": "RANDOM_RBF", "gradual_sine": "SINE", "gradual_waveform": "WAVEFORM"}

df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        ##trees.HoeffdingTree_ADWIN_100000_95_0_AGRAW1_Abrupt
        files_path = glob.glob(f'../results/{drift_key}/{classifier}_{drift["id"]}_{drift_size}_{partialy_drift}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value, metric = metric_name)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,HypothesisTestDetectorKs,HypothesisTestDetectorTt,HypothesisTestDetectorWrs,MultivariateDetector,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2,Studd
generator,,,,,,,,,,,,,,,,,
AGRAW1,115.143667,297.225333,50.394000,29.267667,306.946000,121.919667,72.961333,96.317333,3.588667,115.429630,562.037,128.295333,243.960667,255.534667,147.238667,343.453333,NaN
AGRAW2,88.167391,43.332308,30.546667,29.827586,59.277241,41.058333,76.516333,122.445000,1.650667,69.833462,562.037,44.763000,83.371481,57.705667,24.384333,157.055667,NaN
LED,NaN,NaN,1321.148929,0.000000,NaN,716.170667,95.839000,103.490667,2.052333,NaN,562.037,NaN,NaN,63.509667,46.530714,102.331071,NaN
MIXED,45.699333,21.211000,87.353000,12.915667,12.699333,14.320667,90.302759,113.705000,2.006000,554.148000,562.037,10.534000,21.815333,15.258333,9.500333,82.243333,NaN
RANDOM_RBF,36.327667,41.288000,11.595000,34.902667,18.925000,15.370000,90.860333,79.979333,1.743333,NaN,562.037,24.489000,45.539000,34.210333,6.828667,101.046000,777.690909
SINE,56.192000,29.433333,71.117333,11.268000,16.231333,19.518667,176.185333,78.618333,1.958000,NaN,562.037,16.320000,35.059000,18.824333,12.003333,90.076333,NaN
WAVEFORM,NaN,1579.540000,227.942000,126.517273,NaN,1554.283333,90.758000,128.358000,1.192333,NaN,562.037,889.623636,1560.560000,0.000000,592.612941,0.005000,NaN


# Gradual HT 100K - Percentage no labels: 98% - Metric: delay true detection (average)

In [69]:
classifier = "trees.HoeffdingTree"
drift_size = 100000
#partialy_drift options: 95, 96, 97, 98, 99
partialy_drift = 98
type_drift = "Gradual"
#metric name options: detected changes, detected warnings, delay detection (average), delay true detection (average), MTFA (average), MDR, true changes detected, 
metric_name = "delay true detection (average)"
dict_drift_type = {"gradual_agraw1": "AGRAW1", "gradual_agraw2": "AGRAW2", "gradual_led": "LED", 
                    "gradual_mixed": "MIXED", "gradual_randomRBF": "RANDOM_RBF", "gradual_sine": "SINE", "gradual_waveform": "WAVEFORM"}

df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        ##trees.HoeffdingTree_ADWIN_100000_95_0_AGRAW1_Abrupt
        files_path = glob.glob(f'../results/{drift_key}/{classifier}_{drift["id"]}_{drift_size}_{partialy_drift}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value, metric = metric_name)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,HypothesisTestDetectorKs,HypothesisTestDetectorTt,HypothesisTestDetectorWrs,MultivariateDetector,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2,Studd
generator,,,,,,,,,,,,,,,,,
AGRAW1,199.200667,133.455667,72.653333,31.325000,161.324000,95.295000,68.461667,116.471333,6.466667,58.481923,689.073333,142.079667,211.307000,172.801667,155.671333,391.972667,NaN
AGRAW2,138.118750,55.443200,13.007667,23.082069,35.054800,46.324667,71.370333,78.603667,2.111667,44.805769,689.073333,36.851724,106.360400,227.521379,28.322000,221.152800,NaN
LED,NaN,NaN,324.451364,0.000000,3151.245000,200.915833,87.105000,88.896000,2.346333,NaN,689.073333,NaN,NaN,102.501333,112.667586,123.200690,NaN
MIXED,41.605333,20.140667,13.045000,12.969333,9.856333,12.630333,199.694000,58.429667,2.871667,489.900000,689.073333,12.186333,20.996000,16.827667,11.741333,138.085333,NaN
RANDOM_RBF,43.669667,34.493667,7.496000,27.684333,15.618667,15.972333,48.125862,56.373793,2.921333,NaN,689.073333,26.716333,41.825000,44.610667,20.617333,172.110333,917.616667
SINE,58.353333,21.551333,9.660000,11.502667,17.670000,18.036000,45.503667,63.465517,2.694333,NaN,689.073333,14.038333,31.490667,24.507333,12.735000,135.952333,NaN
WAVEFORM,NaN,1370.800000,160.720667,79.217826,NaN,402.180000,118.225862,127.860000,2.201333,NaN,689.073333,180.564286,1370.800000,138.773333,699.826250,16.942500,NaN


# Gradual HT 100K - Percentage no labels: 99% - Metric: delay true detection (average)

In [70]:
classifier = "trees.HoeffdingTree"
drift_size = 100000
#partialy_drift options: 95, 96, 97, 98, 99
partialy_drift = 99
type_drift = "Gradual"
#metric name options: detected changes, detected warnings, delay detection (average), delay true detection (average), MTFA (average), MDR, true changes detected, 
metric_name = "delay true detection (average)"
dict_drift_type = {"gradual_agraw1": "AGRAW1", "gradual_agraw2": "AGRAW2", "gradual_led": "LED", 
                    "gradual_mixed": "MIXED", "gradual_randomRBF": "RANDOM_RBF", "gradual_sine": "SINE", "gradual_waveform": "WAVEFORM"}

df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        ##trees.HoeffdingTree_ADWIN_100000_95_0_AGRAW1_Abrupt
        files_path = glob.glob(f'../results/{drift_key}/{classifier}_{drift["id"]}_{drift_size}_{partialy_drift}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value, metric = metric_name)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,HypothesisTestDetectorKs,HypothesisTestDetectorTt,HypothesisTestDetectorWrs,MultivariateDetector,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2,Studd
generator,,,,,,,,,,,,,,,,,
AGRAW1,144.465556,122.962667,58.342333,30.486000,94.045333,100.156333,70.195862,109.549333,11.809667,36.580000,801.488571,84.157000,315.827000,285.939000,86.514000,455.313667,NaN
AGRAW2,280.823333,110.962000,26.485357,30.567333,49.565625,46.943462,34.138667,73.800000,4.736667,30.508929,801.488571,45.114545,98.244000,268.318214,92.676333,707.120714,NaN
LED,NaN,NaN,1011.400833,0.000000,NaN,273.042500,79.766000,59.671333,6.908000,NaN,801.488571,NaN,NaN,83.817000,37.374828,460.078621,NaN
MIXED,61.590000,14.772667,4.541667,14.392333,10.917333,13.792333,25.271000,18.941333,6.048667,NaN,801.488571,10.877667,18.061667,24.071667,12.139667,155.589333,NaN
RANDOM_RBF,279.391034,48.765667,10.730667,20.174667,21.136000,23.395000,28.587586,27.132000,6.212000,NaN,801.488571,24.479333,44.203000,39.445000,29.536000,427.680667,51.535
SINE,89.949667,19.461333,5.330333,13.876000,22.526667,15.519333,52.385000,28.123333,5.153000,NaN,801.488571,12.832000,29.641333,28.213000,15.892000,157.680000,NaN
WAVEFORM,NaN,13.680000,246.109200,72.084444,NaN,NaN,86.119655,51.484000,4.566333,NaN,801.488571,94.336667,13.680000,12.470000,10.527273,NaN,NaN


# Abrupt HT 100K - Percentage no labels: 95% - Metric: MTFA (average)

In [71]:
classifier = "trees.HoeffdingTree"
drift_size = 100000
#partialy_drift options: 95, 96, 97, 98, 99
partialy_drift = 95
type_drift = "Abrupt"
#metric name options: detected changes, detected warnings, delay detection (average), delay true detection (average), MTFA (average), MDR, true changes detected, 
metric_name = "MTFA (average)"
dict_drift_type = {"abrupt_agraw1": "AGRAW1", "abrupt_agraw2": "AGRAW2", "abrupt_led": "LED", "abrupt_mixed": "MIXED", "abrupt_randomRBF": "RANDOM_RBF",  
                    "abrupt_sine": "SINE", "abrupt_waveform": "WAVEFORM"}

df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        ##trees.HoeffdingTree_ADWIN_100000_95_0_AGRAW1_Abrupt
        files_path = glob.glob(f'../results/{drift_key}/{classifier}_{drift["id"]}_{drift_size}_{partialy_drift}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value, metric = metric_name)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,HypothesisTestDetectorKs,HypothesisTestDetectorTt,HypothesisTestDetectorWrs,MultivariateDetector,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2,Studd
generator,,,,,,,,,,,,,,,,,
AGRAW1,254.583333,528.289667,511.724333,268.469333,1255.846333,891.992000,266.558333,312.173333,58.383333,266.665357,1015.011667,801.686207,819.374667,990.773333,1210.471667,298.078667,NaN
AGRAW2,956.957241,506.544286,391.652667,443.730667,943.780667,964.039333,449.706000,442.746333,32.556000,266.665357,1015.011667,1005.176333,942.530000,430.638000,420.860667,466.814333,0.500000
LED,NaN,NaN,998.184333,22.634000,0.966667,388.221739,276.671333,323.909667,40.858333,NaN,1015.011667,NaN,0.805000,233.008667,265.270000,292.004000,0.902222
MIXED,814.247333,1013.771667,518.076000,525.268667,1153.473000,920.705000,533.343333,557.923000,36.282667,126.017222,1015.011667,957.088667,1192.679000,364.900333,988.379667,440.629000,NaN
RANDOM_RBF,478.370667,954.512333,715.999667,673.447000,1016.945000,878.851000,411.634333,334.508000,37.477667,NaN,1015.011667,870.049333,1068.710000,165.474667,294.422000,331.923667,53.662857
SINE,1122.875333,694.559000,415.432333,418.451333,860.003000,810.898667,394.331000,469.774000,35.292667,0.890000,1015.011667,785.894000,970.251333,447.071000,746.367000,447.028667,NaN
WAVEFORM,NaN,139.020000,626.605000,244.135217,0.935000,0.886000,337.896000,348.830667,36.974000,NaN,1015.011667,0.713333,0.950000,0.925000,1378.373182,0.950000,NaN


# Abrupt HT 100K - Percentage no labels: 96% - Metric: MTFA (average)

In [72]:
classifier = "trees.HoeffdingTree"
drift_size = 100000
#partialy_drift options: 95, 96, 97, 98, 99
partialy_drift = 96
type_drift = "Abrupt"
#metric name options: detected changes, detected warnings, delay detection (average), delay true detection (average), MTFA (average), MDR, true changes detected, 
metric_name = "MTFA (average)"
dict_drift_type = {"abrupt_agraw1": "AGRAW1", "abrupt_agraw2": "AGRAW2", "abrupt_led": "LED", "abrupt_mixed": "MIXED", "abrupt_randomRBF": "RANDOM_RBF",  
                    "abrupt_sine": "SINE", "abrupt_waveform": "WAVEFORM"}

df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        ##trees.HoeffdingTree_ADWIN_100000_95_0_AGRAW1_Abrupt
        files_path = glob.glob(f'../results/{drift_key}/{classifier}_{drift["id"]}_{drift_size}_{partialy_drift}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value, metric = metric_name)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,HypothesisTestDetectorKs,HypothesisTestDetectorTt,HypothesisTestDetectorWrs,MultivariateDetector,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2,Studd
generator,,,,,,,,,,,,,,,,,
AGRAW1,153.578000,384.094000,492.289667,224.086000,1139.241667,920.959000,234.350333,295.392667,55.848667,278.636333,1067.307,862.495667,763.616667,876.515333,831.138667,200.843667,NaN
AGRAW2,638.488667,424.776000,360.386000,410.614138,767.937667,693.683667,412.126667,374.892667,32.638333,297.190000,1067.307,716.269000,573.741000,361.249000,335.465667,354.783000,NaN
LED,NaN,NaN,832.379000,0.734000,0.730000,645.983750,299.385000,311.883000,42.492333,NaN,1067.307,NaN,0.795667,223.847667,279.685172,279.894828,0.926667
MIXED,750.060333,809.280000,498.787667,468.910000,733.390000,956.990667,509.732667,484.514000,36.462000,0.891667,1067.307,640.018333,633.527667,326.955667,757.315667,336.443333,NaN
RANDOM_RBF,422.031000,731.108333,708.424667,575.443667,984.667000,768.352333,367.221000,310.784000,36.684333,NaN,1067.307,741.220000,693.656333,167.060000,250.966000,331.676333,194.847667
SINE,693.058333,664.387000,334.622000,347.680000,684.039667,874.624667,421.446000,395.291000,34.965000,0.970000,1067.307,528.022333,797.277000,353.617000,494.582000,345.169667,NaN
WAVEFORM,NaN,1602.670000,643.756333,216.337619,0.980000,0.897500,313.343667,338.693333,36.887667,NaN,1067.307,244.432857,0.500000,0.980000,702.976471,0.910000,NaN


# Abrupt HT 100K - Percentage no labels: 97% - Metric: MTFA (average)

In [73]:
classifier = "trees.HoeffdingTree"
drift_size = 100000
#partialy_drift options: 95, 96, 97, 98, 99
partialy_drift = 97
type_drift = "Abrupt"
#metric name options: detected changes, detected warnings, delay detection (average), delay true detection (average), MTFA (average), MDR, true changes detected, 
metric_name = "MTFA (average)"
dict_drift_type = {"abrupt_agraw1": "AGRAW1", "abrupt_agraw2": "AGRAW2", "abrupt_led": "LED", "abrupt_mixed": "MIXED", "abrupt_randomRBF": "RANDOM_RBF",  
                    "abrupt_sine": "SINE", "abrupt_waveform": "WAVEFORM"}

df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        ##trees.HoeffdingTree_ADWIN_100000_95_0_AGRAW1_Abrupt
        files_path = glob.glob(f'../results/{drift_key}/{classifier}_{drift["id"]}_{drift_size}_{partialy_drift}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value, metric = metric_name)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,HypothesisTestDetectorKs,HypothesisTestDetectorTt,HypothesisTestDetectorWrs,MultivariateDetector,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2,Studd
generator,,,,,,,,,,,,,,,,,
AGRAW1,211.819286,329.634667,491.977000,253.649667,423.896552,926.740000,248.739000,255.627667,56.841000,276.757407,951.028333,726.519000,499.505333,692.543333,938.669333,486.676667,NaN
AGRAW2,416.245000,339.437083,308.636667,290.394828,508.905333,565.319333,388.817333,392.052333,32.844333,276.757407,951.028333,500.891000,442.258750,261.560667,282.376667,324.746000,NaN
LED,NaN,NaN,960.661034,0.000000,0.920000,427.777500,279.938667,335.906667,42.498333,NaN,951.028333,NaN,0.889333,214.695333,238.736897,359.442593,0.960000
MIXED,592.817667,501.043000,363.023667,398.554667,648.331333,642.116667,441.487000,389.046667,36.761000,0.936667,951.028333,538.046333,516.092000,251.377000,495.444000,285.560000,NaN
RANDOM_RBF,402.965333,580.953333,461.998000,433.002667,562.819000,462.469333,258.688333,240.315667,35.973667,NaN,951.028333,549.849333,493.395333,185.930667,194.648333,302.788000,96.451053
SINE,654.870667,444.758667,332.345333,271.701333,593.053667,571.959000,436.048333,327.593667,35.247333,0.970000,951.028333,466.910667,591.789000,270.509667,422.669667,323.015333,NaN
WAVEFORM,NaN,NaN,499.744333,214.991111,0.950000,0.915000,394.598667,337.512333,36.591667,NaN,951.028333,427.060000,NaN,19.361667,440.786667,137.220000,NaN


# Abrupt HT 100K - Percentage no labels: 98% - Metric: MTFA (average)

In [74]:
classifier = "trees.HoeffdingTree"
drift_size = 100000
#partialy_drift options: 95, 96, 97, 98, 99
partialy_drift = 98
type_drift = "Abrupt"
#metric name options: detected changes, detected warnings, delay detection (average), delay true detection (average), MTFA (average), MDR, true changes detected, 
metric_name = "MTFA (average)"
dict_drift_type = {"abrupt_agraw1": "AGRAW1", "abrupt_agraw2": "AGRAW2", "abrupt_led": "LED", "abrupt_mixed": "MIXED", "abrupt_randomRBF": "RANDOM_RBF",  
                    "abrupt_sine": "SINE", "abrupt_waveform": "WAVEFORM"}

df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        ##trees.HoeffdingTree_ADWIN_100000_95_0_AGRAW1_Abrupt
        files_path = glob.glob(f'../results/{drift_key}/{classifier}_{drift["id"]}_{drift_size}_{partialy_drift}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value, metric = metric_name)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,HypothesisTestDetectorKs,HypothesisTestDetectorTt,HypothesisTestDetectorWrs,MultivariateDetector,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2,Studd
generator,,,,,,,,,,,,,,,,,
AGRAW1,85.980333,189.362667,313.433667,189.780333,237.038333,353.166000,249.916000,221.302000,59.286667,218.539286,286.261,448.694000,337.038333,261.336000,540.191000,155.007333,NaN
AGRAW2,237.730000,242.925909,254.305333,230.349655,351.827143,429.539333,291.493333,353.701667,34.331667,218.539286,286.261,305.305000,361.774091,235.365667,194.973333,184.462500,NaN
LED,NaN,NaN,87.009167,18.176000,0.960000,88.494444,220.551667,268.308333,41.363333,NaN,286.261,NaN,0.928276,197.902333,209.447857,265.869000,NaN
MIXED,362.817000,454.159667,312.559000,317.304667,415.817000,397.080333,343.749667,325.001000,36.364000,0.670000,286.261,370.796000,408.577000,202.002333,334.296333,368.995333,NaN
RANDOM_RBF,279.446333,376.896000,335.562000,322.739333,368.197667,376.488333,251.487667,252.765000,38.039000,NaN,286.261,414.729667,397.371667,168.050000,161.388667,423.977667,0.860556
SINE,372.406333,386.137667,236.736667,263.667333,361.969333,331.620667,371.102333,362.927000,35.230667,NaN,286.261,330.911333,484.291333,210.265667,404.095000,360.902000,NaN
WAVEFORM,NaN,NaN,518.221667,156.850000,NaN,0.976667,335.800333,363.436667,36.202333,NaN,286.261,0.818750,NaN,42.267500,80.063636,0.976667,NaN


# Abrupt HT 100K - Percentage no labels: 99% - Metric: MTFA (average)

In [75]:
classifier = "trees.HoeffdingTree"
drift_size = 100000
#partialy_drift options: 95, 96, 97, 98, 99
partialy_drift = 99
type_drift = "Abrupt"
#metric name options: detected changes, detected warnings, delay detection (average), delay true detection (average), MTFA (average), MDR, true changes detected, 
metric_name = "MTFA (average)"
dict_drift_type = {"abrupt_agraw1": "AGRAW1", "abrupt_agraw2": "AGRAW2", "abrupt_led": "LED", "abrupt_mixed": "MIXED", "abrupt_randomRBF": "RANDOM_RBF",  
                    "abrupt_sine": "SINE", "abrupt_waveform": "WAVEFORM"}

df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        ##trees.HoeffdingTree_ADWIN_100000_95_0_AGRAW1_Abrupt
        files_path = glob.glob(f'../results/{drift_key}/{classifier}_{drift["id"]}_{drift_size}_{partialy_drift}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value, metric = metric_name)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,HypothesisTestDetectorKs,HypothesisTestDetectorTt,HypothesisTestDetectorWrs,MultivariateDetector,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2,Studd
generator,,,,,,,,,,,,,,,,,
AGRAW1,6.728235,90.446552,382.443000,94.327000,89.304667,165.048333,196.458333,100.243000,54.217333,133.375000,0.915,132.597667,26.365862,59.124667,79.209000,35.999667,NaN
AGRAW2,1.000000,125.719091,149.512000,187.715862,67.935500,204.560000,120.355862,170.331333,31.221667,131.621538,0.915,198.810952,152.300000,98.315000,153.044483,63.096250,NaN
LED,NaN,NaN,0.966923,0.988333,NaN,0.970000,170.732759,167.330345,43.013333,NaN,0.915,NaN,0.970000,166.154000,202.348148,168.991034,NaN
MIXED,214.873333,171.300667,168.598333,158.098000,227.075000,176.861333,231.795000,208.265000,39.982667,NaN,0.915,195.559000,197.319333,122.507333,173.350333,205.509667,NaN
RANDOM_RBF,97.359655,198.409333,198.285000,186.893667,159.793000,204.177000,494.185333,147.114138,38.438000,NaN,0.915,187.443333,219.748333,195.078000,121.639333,136.933667,0.9925
SINE,209.652667,225.518000,218.269667,181.133000,242.183000,218.350667,192.596333,198.158276,36.014000,NaN,0.915,259.583667,286.756667,124.840333,189.716000,219.102000,NaN
WAVEFORM,NaN,NaN,360.247727,162.645000,NaN,0.990000,153.177586,203.116552,37.356667,NaN,0.915,0.990000,NaN,0.946667,7.592500,88.185000,NaN


# Gradual HT 100K - Percentage no labels: 95% - Metric: MTFA (average)

In [76]:
classifier = "trees.HoeffdingTree"
drift_size = 100000
#partialy_drift options: 95, 96, 97, 98, 99
partialy_drift = 95
type_drift = "Gradual"
#metric name options: detected changes, detected warnings, delay detection (average), delay true detection (average), MTFA (average), MDR, true changes detected, 
metric_name = "MTFA (average)"
dict_drift_type = {"gradual_agraw1": "AGRAW1", "gradual_agraw2": "AGRAW2", "gradual_led": "LED", 
                    "gradual_mixed": "MIXED", "gradual_randomRBF": "RANDOM_RBF", "gradual_sine": "SINE", "gradual_waveform": "WAVEFORM"}

df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        ##trees.HoeffdingTree_ADWIN_100000_95_0_AGRAW1_Abrupt
        files_path = glob.glob(f'../results/{drift_key}/{classifier}_{drift["id"]}_{drift_size}_{partialy_drift}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value, metric = metric_name)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,HypothesisTestDetectorKs,HypothesisTestDetectorTt,HypothesisTestDetectorWrs,MultivariateDetector,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2,Studd
generator,,,,,,,,,,,,,,,,,
AGRAW1,373.433667,483.351786,496.483667,258.494667,992.620667,953.271000,267.899000,335.512667,56.357333,431.288214,1015.011667,881.639000,887.677241,1640.059333,862.966207,365.785000,NaN
AGRAW2,704.753103,496.218966,372.941000,443.319667,933.760333,757.790333,441.476333,403.686000,32.582667,431.288214,1015.011667,1272.318333,951.967000,459.997333,422.211000,444.806000,NaN
LED,NaN,NaN,1021.140667,22.634000,0.950000,115.590000,268.815000,341.444000,41.806000,NaN,1015.011667,NaN,NaN,226.782000,342.248667,295.838333,0.670
MIXED,797.893333,874.520000,545.861333,587.150333,1037.048333,921.469000,827.548667,603.316667,37.369000,1973.535000,1015.011667,1029.263333,849.749000,369.263000,838.706000,432.235333,NaN
RANDOM_RBF,467.993000,941.406667,771.227000,744.524667,1045.530333,1141.768333,383.042667,418.008667,37.376333,NaN,1015.011667,888.310333,871.595000,178.407667,314.947333,352.843667,460.688
SINE,1039.357333,692.483333,441.080000,360.037667,1039.926667,1084.701333,593.889000,505.292667,34.729667,NaN,1015.011667,773.280667,864.271333,442.139000,697.706667,447.703000,NaN
WAVEFORM,NaN,0.936667,642.325000,256.695000,0.856667,0.722500,396.158000,333.313000,36.899667,NaN,1015.011667,308.112500,0.886000,179.447500,777.409167,0.950000,0.800


# Gradual HT 100K - Percentage no labels: 96% - Metric: MTFA (average)

In [77]:
classifier = "trees.HoeffdingTree"
drift_size = 100000
#partialy_drift options: 95, 96, 97, 98, 99
partialy_drift = 96
type_drift = "Gradual"
#metric name options: detected changes, detected warnings, delay detection (average), delay true detection (average), MTFA (average), MDR, true changes detected, 
metric_name = "MTFA (average)"
dict_drift_type = {"gradual_agraw1": "AGRAW1", "gradual_agraw2": "AGRAW2", "gradual_led": "LED", 
                    "gradual_mixed": "MIXED", "gradual_randomRBF": "RANDOM_RBF", "gradual_sine": "SINE", "gradual_waveform": "WAVEFORM"}

df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        ##trees.HoeffdingTree_ADWIN_100000_95_0_AGRAW1_Abrupt
        files_path = glob.glob(f'../results/{drift_key}/{classifier}_{drift["id"]}_{drift_size}_{partialy_drift}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value, metric = metric_name)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,HypothesisTestDetectorKs,HypothesisTestDetectorTt,HypothesisTestDetectorWrs,MultivariateDetector,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2,Studd
generator,,,,,,,,,,,,,,,,,
AGRAW1,311.708333,513.584333,517.659667,192.556667,737.212333,880.243333,302.317333,379.096000,55.673667,1012.503793,1067.307,1222.781000,658.106000,873.777333,810.476667,255.773000,NaN
AGRAW2,436.579667,392.020667,361.090667,461.014828,846.696000,718.771000,429.080667,499.659000,32.833000,1007.480000,1067.307,681.535667,708.932000,346.614333,331.273000,367.549000,NaN
LED,NaN,NaN,713.745714,0.734000,0.780000,954.971176,316.158000,302.667667,44.144333,NaN,1067.307,NaN,NaN,257.627667,530.653793,309.034483,NaN
MIXED,749.252333,642.841667,488.069000,474.315000,938.506000,943.371000,786.025333,613.563000,37.068000,273.823846,1067.307,841.916667,742.297667,280.642333,651.882667,332.134667,NaN
RANDOM_RBF,310.681333,851.099333,689.549000,573.086000,631.504667,788.394333,270.336667,374.240333,37.086333,NaN,1067.307,819.476667,700.297667,186.390333,268.024000,321.665000,51.243667
SINE,728.016667,598.271000,385.184000,394.572667,796.754667,722.675667,471.802000,415.579333,34.837333,NaN,1067.307,634.086000,719.161000,369.484667,538.013333,334.599000,NaN
WAVEFORM,NaN,589.120000,461.738000,196.192500,0.920000,0.900000,373.890000,358.423333,36.910667,NaN,1067.307,276.015714,18.666000,1056.277500,663.572778,0.910000,NaN


# Gradual HT 100K - Percentage no labels: 97% - Metric: MTFA (average)

In [78]:
classifier = "trees.HoeffdingTree"
drift_size = 100000
#partialy_drift options: 95, 96, 97, 98, 99
partialy_drift = 97
type_drift = "Gradual"
#metric name options: detected changes, detected warnings, delay detection (average), delay true detection (average), MTFA (average), MDR, true changes detected, 
metric_name = "MTFA (average)"
dict_drift_type = {"gradual_agraw1": "AGRAW1", "gradual_agraw2": "AGRAW2", "gradual_led": "LED", 
                    "gradual_mixed": "MIXED", "gradual_randomRBF": "RANDOM_RBF", "gradual_sine": "SINE", "gradual_waveform": "WAVEFORM"}

df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        ##trees.HoeffdingTree_ADWIN_100000_95_0_AGRAW1_Abrupt
        files_path = glob.glob(f'../results/{drift_key}/{classifier}_{drift["id"]}_{drift_size}_{partialy_drift}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value, metric = metric_name)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,HypothesisTestDetectorKs,HypothesisTestDetectorTt,HypothesisTestDetectorWrs,MultivariateDetector,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2,Studd
generator,,,,,,,,,,,,,,,,,
AGRAW1,167.259000,318.537931,371.934333,177.146667,837.667586,708.734333,250.891000,299.618667,55.946000,416.195185,951.028333,526.081667,365.527241,853.481000,691.811333,283.917333,NaN
AGRAW2,281.988696,304.873462,359.752667,318.585517,573.647241,500.329333,391.443333,401.670000,33.924333,391.006538,951.028333,528.832000,484.166296,286.597000,245.912667,375.886333,NaN
LED,NaN,NaN,623.162308,0.000000,NaN,216.042000,287.557333,298.904000,41.153333,NaN,951.028333,NaN,NaN,216.987667,273.277857,336.401429,NaN
MIXED,626.850333,571.042333,418.871000,441.486667,680.846333,558.365000,377.794483,586.891667,37.915000,0.922000,951.028333,497.412333,569.857667,246.006000,652.820333,275.359667,NaN
RANDOM_RBF,338.321000,593.749333,640.859667,511.301667,629.642333,546.060667,338.667000,340.215000,37.986333,NaN,951.028333,544.939333,599.540333,177.653333,199.104333,332.401667,33.984545
SINE,601.997000,486.406667,482.498333,335.929000,556.187333,574.059333,495.695667,345.536333,35.752000,NaN,951.028333,572.516667,587.052333,279.984000,622.745667,312.321333,NaN
WAVEFORM,NaN,526.720000,548.640667,154.560455,NaN,0.870000,346.295667,405.074667,35.532667,NaN,951.028333,232.836364,0.885000,0.965000,199.080000,34.685000,NaN


# Gradual HT 100K - Percentage no labels: 98% - Metric: MTFA (average)

In [79]:
classifier = "trees.HoeffdingTree"
drift_size = 100000
#partialy_drift options: 95, 96, 97, 98, 99
partialy_drift = 98
type_drift = "Gradual"
#metric name options: detected changes, detected warnings, delay detection (average), delay true detection (average), MTFA (average), MDR, true changes detected, 
metric_name = "MTFA (average)"
dict_drift_type = {"gradual_agraw1": "AGRAW1", "gradual_agraw2": "AGRAW2", "gradual_led": "LED", 
                    "gradual_mixed": "MIXED", "gradual_randomRBF": "RANDOM_RBF", "gradual_sine": "SINE", "gradual_waveform": "WAVEFORM"}

df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        ##trees.HoeffdingTree_ADWIN_100000_95_0_AGRAW1_Abrupt
        files_path = glob.glob(f'../results/{drift_key}/{classifier}_{drift["id"]}_{drift_size}_{partialy_drift}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value, metric = metric_name)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,HypothesisTestDetectorKs,HypothesisTestDetectorTt,HypothesisTestDetectorWrs,MultivariateDetector,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2,Studd
generator,,,,,,,,,,,,,,,,,
AGRAW1,47.148667,246.069000,379.463333,168.205333,220.762414,404.244333,210.784667,295.939000,56.361000,222.083077,286.261,516.994667,331.654667,403.462333,772.653667,144.363333,NaN
AGRAW2,487.046250,210.072000,238.346667,236.183448,243.844000,408.346333,350.883000,338.848333,32.284667,189.802308,286.261,302.645172,346.530000,174.228276,175.390000,304.667600,NaN
LED,NaN,NaN,187.839545,18.176000,0.495000,0.931667,280.022667,236.563000,40.777000,NaN,286.261,NaN,NaN,299.521000,692.136897,277.619655,NaN
MIXED,352.119667,424.061333,318.423333,282.566667,340.481000,350.410000,340.057000,286.854333,39.427000,0.953333,286.261,413.832000,388.381667,181.016333,397.790333,332.115333,NaN
RANDOM_RBF,236.149667,364.104000,350.743000,329.878000,359.953000,359.728000,212.245517,246.697931,39.190667,NaN,286.261,423.171000,401.280667,176.973667,187.715667,359.532667,0.891667
SINE,395.929667,311.510000,259.345333,250.835667,385.846667,414.231000,309.765000,303.945517,35.927333,NaN,286.261,399.117667,441.147000,216.450667,366.608000,302.145333,NaN
WAVEFORM,NaN,0.890000,389.443333,194.045217,NaN,0.970000,319.009655,354.672000,37.368000,NaN,286.261,0.964286,0.890000,267.210000,319.092500,33.000000,NaN


# Gradual HT 100K - Percentage no labels: 99% - Metric: MTFA (average)

In [80]:
classifier = "trees.HoeffdingTree"
drift_size = 100000
#partialy_drift options: 95, 96, 97, 98, 99
partialy_drift = 99
type_drift = "Gradual"
#metric name options: detected changes, detected warnings, delay detection (average), delay true detection (average), MTFA (average), MDR, true changes detected, 
metric_name = "MTFA (average)"
dict_drift_type = {"gradual_agraw1": "AGRAW1", "gradual_agraw2": "AGRAW2", "gradual_led": "LED", 
                    "gradual_mixed": "MIXED", "gradual_randomRBF": "RANDOM_RBF", "gradual_sine": "SINE", "gradual_waveform": "WAVEFORM"}

df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        ##trees.HoeffdingTree_ADWIN_100000_95_0_AGRAW1_Abrupt
        files_path = glob.glob(f'../results/{drift_key}/{classifier}_{drift["id"]}_{drift_size}_{partialy_drift}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value, metric = metric_name)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,HypothesisTestDetectorKs,HypothesisTestDetectorTt,HypothesisTestDetectorWrs,MultivariateDetector,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2,Studd
generator,,,,,,,,,,,,,,,,,
AGRAW1,49.141111,88.235172,148.814667,130.821333,87.556667,287.326000,163.638966,228.402333,57.500000,98.798214,0.915,195.460667,57.970357,14.052000,162.544667,47.526000,NaN
AGRAW2,0.976667,153.150000,172.721786,164.437667,124.613750,195.802692,181.597667,180.702143,33.591333,25.973929,0.915,193.665909,183.113000,170.973214,106.176000,49.747143,NaN
LED,NaN,NaN,5.845455,0.988333,NaN,14.177500,228.950333,188.498000,44.186333,NaN,0.915,NaN,NaN,182.143333,170.891379,181.487931,NaN
MIXED,208.643000,180.045000,172.016333,203.115333,190.834333,174.254333,217.415000,188.755667,37.995667,NaN,0.915,205.571000,192.900000,109.348333,151.183333,198.038667,NaN
RANDOM_RBF,29.134138,242.533333,194.215000,173.016000,210.403000,202.146000,163.611379,153.205667,40.328333,NaN,0.915,196.789000,170.746667,184.653333,121.673000,143.186000,0.995
SINE,252.612333,187.817333,181.767667,176.652667,282.768667,181.667333,179.792667,141.565333,34.199667,NaN,0.915,182.903667,215.476000,126.697000,194.171333,203.833000,NaN
WAVEFORM,NaN,0.990000,186.545600,141.515000,NaN,NaN,131.738621,122.767667,35.822333,NaN,0.915,0.986667,0.990000,0.953333,13.693636,NaN,NaN


# Abrupt HT 100K - Percentage no labels: 95% - Metric: MDR

In [81]:
classifier = "trees.HoeffdingTree"
drift_size = 100000
#partialy_drift options: 95, 96, 97, 98, 99
partialy_drift = 95
type_drift = "Abrupt"
#metric name options: detected changes, detected warnings, delay detection (average), delay true detection (average), MTFA (average), MDR, true changes detected, 
metric_name = "MDR"
dict_drift_type = {"abrupt_agraw1": "AGRAW1", "abrupt_agraw2": "AGRAW2", "abrupt_led": "LED", "abrupt_mixed": "MIXED", "abrupt_randomRBF": "RANDOM_RBF",  
                    "abrupt_sine": "SINE", "abrupt_waveform": "WAVEFORM"}

df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        ##trees.HoeffdingTree_ADWIN_100000_95_0_AGRAW1_Abrupt
        files_path = glob.glob(f'../results/{drift_key}/{classifier}_{drift["id"]}_{drift_size}_{partialy_drift}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value, metric = metric_name)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,HypothesisTestDetectorKs,HypothesisTestDetectorTt,HypothesisTestDetectorWrs,MultivariateDetector,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2,Studd
generator,,,,,,,,,,,,,,,,,
AGRAW1,0.641667,0.508333,0.108333,0.258333,0.350000,0.116667,0.008333,0.016667,0.0,0.916667,0.125,0.100000,0.391667,0.300000,0.316667,0.641667,1.000000
AGRAW2,0.066667,0.075000,0.083333,0.083333,0.008333,0.016667,0.158333,0.158333,0.0,0.916667,0.125,0.016667,0.058333,0.000000,0.016667,0.041667,0.991667
LED,1.000000,1.000000,0.325000,1.000000,0.975000,0.766667,0.033333,0.050000,0.0,1.000000,0.125,1.000000,0.750000,0.066667,0.725000,0.200000,0.925000
MIXED,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.066667,0.066667,0.0,0.825000,0.125,0.008333,0.000000,0.000000,0.000000,0.000000,1.000000
RANDOM_RBF,0.000000,0.000000,0.058333,0.000000,0.000000,0.000000,0.050000,0.033333,0.0,1.000000,0.125,0.000000,0.000000,0.000000,0.000000,0.016667,0.733333
SINE,0.000000,0.000000,0.050000,0.033333,0.000000,0.000000,0.066667,0.058333,0.0,0.991667,0.125,0.000000,0.000000,0.000000,0.000000,0.008333,1.000000
WAVEFORM,1.000000,0.991667,0.108333,0.891667,0.991667,0.958333,0.108333,0.108333,0.0,1.000000,0.125,0.966667,0.991667,1.000000,0.758333,1.000000,1.000000


# Abrupt HT 100K - Percentage no labels: 96% - Metric: MDR

In [82]:
classifier = "trees.HoeffdingTree"
drift_size = 100000
#partialy_drift options: 95, 96, 97, 98, 99
partialy_drift = 96
type_drift = "Abrupt"
#metric name options: detected changes, detected warnings, delay detection (average), delay true detection (average), MTFA (average), MDR, true changes detected, 
metric_name = "MDR"
dict_drift_type = {"abrupt_agraw1": "AGRAW1", "abrupt_agraw2": "AGRAW2", "abrupt_led": "LED", "abrupt_mixed": "MIXED", "abrupt_randomRBF": "RANDOM_RBF",  
                    "abrupt_sine": "SINE", "abrupt_waveform": "WAVEFORM"}

df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        ##trees.HoeffdingTree_ADWIN_100000_95_0_AGRAW1_Abrupt
        files_path = glob.glob(f'../results/{drift_key}/{classifier}_{drift["id"]}_{drift_size}_{partialy_drift}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value, metric = metric_name)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,HypothesisTestDetectorKs,HypothesisTestDetectorTt,HypothesisTestDetectorWrs,MultivariateDetector,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2,Studd
generator,,,,,,,,,,,,,,,,,
AGRAW1,0.658333,0.483333,0.100000,0.325000,0.450000,0.125000,0.058333,0.083333,0.0,0.883333,0.125,0.225000,0.575000,0.400000,0.341667,0.691667,1.000000
AGRAW2,0.133333,0.016667,0.066667,0.116667,0.016667,0.016667,0.225000,0.125000,0.0,0.891667,0.125,0.008333,0.033333,0.008333,0.008333,0.058333,1.000000
LED,1.000000,1.000000,0.458333,1.000000,0.991667,0.833333,0.100000,0.091667,0.0,1.000000,0.125,1.000000,0.750000,0.041667,0.733333,0.183333,0.950000
MIXED,0.008333,0.000000,0.025000,0.000000,0.000000,0.000000,0.066667,0.058333,0.0,0.900000,0.125,0.008333,0.000000,0.000000,0.000000,0.000000,1.000000
RANDOM_RBF,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.083333,0.058333,0.0,1.000000,0.125,0.000000,0.000000,0.000000,0.000000,0.033333,0.708333
SINE,0.000000,0.008333,0.025000,0.025000,0.000000,0.008333,0.025000,0.058333,0.0,0.991667,0.125,0.016667,0.000000,0.000000,0.000000,0.000000,1.000000
WAVEFORM,1.000000,0.991667,0.158333,0.775000,0.991667,0.966667,0.183333,0.141667,0.0,1.000000,0.125,0.975000,0.991667,1.000000,0.841667,1.000000,1.000000


# Abrupt HT 100K - Percentage no labels: 97% - Metric: MDR

In [83]:
classifier = "trees.HoeffdingTree"
drift_size = 100000
#partialy_drift options: 95, 96, 97, 98, 99
partialy_drift = 97
type_drift = "Abrupt"
#metric name options: detected changes, detected warnings, delay detection (average), delay true detection (average), MTFA (average), MDR, true changes detected, 
metric_name = "MDR"
dict_drift_type = {"abrupt_agraw1": "AGRAW1", "abrupt_agraw2": "AGRAW2", "abrupt_led": "LED", "abrupt_mixed": "MIXED", "abrupt_randomRBF": "RANDOM_RBF",  
                    "abrupt_sine": "SINE", "abrupt_waveform": "WAVEFORM"}

df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        ##trees.HoeffdingTree_ADWIN_100000_95_0_AGRAW1_Abrupt
        files_path = glob.glob(f'../results/{drift_key}/{classifier}_{drift["id"]}_{drift_size}_{partialy_drift}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value, metric = metric_name)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,HypothesisTestDetectorKs,HypothesisTestDetectorTt,HypothesisTestDetectorWrs,MultivariateDetector,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2,Studd
generator,,,,,,,,,,,,,,,,,
AGRAW1,0.666667,0.525000,0.191667,0.250000,0.541667,0.216667,0.083333,0.108333,0.0,0.950000,0.383333,0.275000,0.566667,0.475000,0.383333,0.691667,1.000000
AGRAW2,0.466667,0.316667,0.116667,0.183333,0.058333,0.008333,0.266667,0.275000,0.0,0.950000,0.383333,0.000000,0.266667,0.083333,0.033333,0.141667,1.000000
LED,1.000000,1.000000,0.558333,1.000000,0.975000,0.883333,0.150000,0.225000,0.0,1.000000,0.383333,1.000000,0.750000,0.075000,0.591667,0.333333,0.983333
MIXED,0.000000,0.000000,0.058333,0.000000,0.000000,0.000000,0.158333,0.091667,0.0,0.966667,0.383333,0.000000,0.000000,0.008333,0.000000,0.000000,1.000000
RANDOM_RBF,0.000000,0.000000,0.033333,0.000000,0.008333,0.000000,0.050000,0.141667,0.0,1.000000,0.383333,0.000000,0.000000,0.000000,0.016667,0.033333,0.791667
SINE,0.008333,0.008333,0.050000,0.016667,0.016667,0.016667,0.091667,0.083333,0.0,0.991667,0.383333,0.008333,0.008333,0.008333,0.008333,0.066667,1.000000
WAVEFORM,1.000000,1.000000,0.258333,0.733333,0.983333,0.983333,0.291667,0.183333,0.0,1.000000,0.383333,0.958333,1.000000,0.975000,0.925000,0.983333,1.000000


# Abrupt HT 100K - Percentage no labels: 98% - Metric: MDR

In [84]:
classifier = "trees.HoeffdingTree"
drift_size = 100000
#partialy_drift options: 95, 96, 97, 98, 99
partialy_drift = 98
type_drift = "Abrupt"
#metric name options: detected changes, detected warnings, delay detection (average), delay true detection (average), MTFA (average), MDR, true changes detected, 
metric_name = "MDR"
dict_drift_type = {"abrupt_agraw1": "AGRAW1", "abrupt_agraw2": "AGRAW2", "abrupt_led": "LED", "abrupt_mixed": "MIXED", "abrupt_randomRBF": "RANDOM_RBF",  
                    "abrupt_sine": "SINE", "abrupt_waveform": "WAVEFORM"}

df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        ##trees.HoeffdingTree_ADWIN_100000_95_0_AGRAW1_Abrupt
        files_path = glob.glob(f'../results/{drift_key}/{classifier}_{drift["id"]}_{drift_size}_{partialy_drift}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value, metric = metric_name)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,HypothesisTestDetectorKs,HypothesisTestDetectorTt,HypothesisTestDetectorWrs,MultivariateDetector,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2,Studd
generator,,,,,,,,,,,,,,,,,
AGRAW1,0.716667,0.591667,0.233333,0.191667,0.566667,0.325000,0.216667,0.183333,0.0,0.850000,0.65,0.433333,0.641667,0.591667,0.516667,0.683333,1.00
AGRAW2,0.916667,0.366667,0.158333,0.100000,0.200000,0.075000,0.308333,0.325000,0.0,0.850000,0.65,0.058333,0.383333,0.291667,0.075000,0.358333,1.00
LED,1.000000,1.000000,0.783333,1.000000,0.991667,0.933333,0.233333,0.208333,0.0,1.000000,0.65,1.000000,0.750000,0.158333,0.550000,0.166667,1.00
MIXED,0.008333,0.000000,0.041667,0.000000,0.000000,0.000000,0.133333,0.091667,0.0,0.991667,0.65,0.000000,0.000000,0.008333,0.000000,0.100000,1.00
RANDOM_RBF,0.016667,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.191667,0.0,1.000000,0.65,0.000000,0.000000,0.008333,0.008333,0.158333,0.85
SINE,0.025000,0.025000,0.066667,0.066667,0.050000,0.033333,0.166667,0.175000,0.0,1.000000,0.65,0.008333,0.033333,0.058333,0.025000,0.116667,1.00
WAVEFORM,1.000000,1.000000,0.425000,0.800000,1.000000,0.975000,0.416667,0.350000,0.0,1.000000,0.65,0.958333,1.000000,0.966667,0.925000,0.991667,1.00


# Abrupt HT 100K - Percentage no labels: 99% - Metric: MDR

In [85]:
classifier = "trees.HoeffdingTree"
drift_size = 100000
#partialy_drift options: 95, 96, 97, 98, 99
partialy_drift = 99
type_drift = "Abrupt"
#metric name options: detected changes, detected warnings, delay detection (average), delay true detection (average), MTFA (average), MDR, true changes detected, 
metric_name = "MDR"
dict_drift_type = {"abrupt_agraw1": "AGRAW1", "abrupt_agraw2": "AGRAW2", "abrupt_led": "LED", "abrupt_mixed": "MIXED", "abrupt_randomRBF": "RANDOM_RBF",  
                    "abrupt_sine": "SINE", "abrupt_waveform": "WAVEFORM"}

df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        ##trees.HoeffdingTree_ADWIN_100000_95_0_AGRAW1_Abrupt
        files_path = glob.glob(f'../results/{drift_key}/{classifier}_{drift["id"]}_{drift_size}_{partialy_drift}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value, metric = metric_name)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,HypothesisTestDetectorKs,HypothesisTestDetectorTt,HypothesisTestDetectorWrs,MultivariateDetector,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2,Studd
generator,,,,,,,,,,,,,,,,,
AGRAW1,0.850000,0.658333,0.533333,0.208333,0.675000,0.491667,0.466667,0.508333,0.016667,0.833333,0.883333,0.533333,0.716667,0.708333,0.658333,0.700000,1.000000
AGRAW2,0.991667,0.833333,0.308333,0.250000,0.725000,0.533333,0.500000,0.416667,0.000000,0.841667,0.883333,0.516667,0.825000,0.558333,0.358333,0.800000,1.000000
LED,1.000000,1.000000,0.891667,1.000000,1.000000,0.975000,0.458333,0.475000,0.008333,1.000000,0.883333,1.000000,0.750000,0.333333,0.616667,0.433333,1.000000
MIXED,0.000000,0.000000,0.033333,0.008333,0.000000,0.000000,0.183333,0.158333,0.000000,1.000000,0.883333,0.016667,0.000000,0.066667,0.008333,0.366667,1.000000
RANDOM_RBF,0.683333,0.000000,0.008333,0.008333,0.025000,0.000000,0.308333,0.283333,0.000000,1.000000,0.883333,0.000000,0.016667,0.408333,0.125000,0.541667,0.966667
SINE,0.141667,0.150000,0.091667,0.083333,0.116667,0.116667,0.233333,0.291667,0.000000,1.000000,0.883333,0.091667,0.166667,0.125000,0.083333,0.400000,1.000000
WAVEFORM,1.000000,1.000000,0.708333,0.708333,1.000000,0.991667,0.525000,0.608333,0.000000,1.000000,0.883333,0.966667,1.000000,0.991667,0.966667,0.975000,1.000000


# Gradual HT 100K - Percentage no labels: 95% - Metric: MDR

In [86]:
classifier = "trees.HoeffdingTree"
drift_size = 100000
#partialy_drift options: 95, 96, 97, 98, 99
partialy_drift = 95
type_drift = "Gradual"
#metric name options: detected changes, detected warnings, delay detection (average), delay true detection (average), MTFA (average), MDR, true changes detected, 
metric_name = "MDR"
dict_drift_type = {"gradual_agraw1": "AGRAW1", "gradual_agraw2": "AGRAW2", "gradual_led": "LED", 
                    "gradual_mixed": "MIXED", "gradual_randomRBF": "RANDOM_RBF", "gradual_sine": "SINE", "gradual_waveform": "WAVEFORM"}

df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        ##trees.HoeffdingTree_ADWIN_100000_95_0_AGRAW1_Abrupt
        files_path = glob.glob(f'../results/{drift_key}/{classifier}_{drift["id"]}_{drift_size}_{partialy_drift}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value, metric = metric_name)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,HypothesisTestDetectorKs,HypothesisTestDetectorTt,HypothesisTestDetectorWrs,MultivariateDetector,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2,Studd
generator,,,,,,,,,,,,,,,,,
AGRAW1,0.633333,0.458333,0.091667,0.241667,0.391667,0.083333,0.016667,0.033333,0.0,0.916667,0.125,0.150000,0.458333,0.383333,0.266667,0.616667,1.000000
AGRAW2,0.116667,0.058333,0.108333,0.125000,0.008333,0.016667,0.166667,0.241667,0.0,0.916667,0.125,0.016667,0.025000,0.000000,0.066667,0.033333,1.000000
LED,1.000000,1.000000,0.383333,1.000000,0.991667,0.850000,0.016667,0.033333,0.0,1.000000,0.125,1.000000,1.000000,0.025000,0.650000,0.216667,0.991667
MIXED,0.000000,0.016667,0.083333,0.050000,0.000000,0.000000,0.250000,0.191667,0.0,0.841667,0.125,0.025000,0.000000,0.000000,0.058333,0.000000,1.000000
RANDOM_RBF,0.000000,0.000000,0.141667,0.025000,0.000000,0.000000,0.091667,0.033333,0.0,1.000000,0.125,0.000000,0.000000,0.000000,0.000000,0.008333,0.683333
SINE,0.000000,0.016667,0.058333,0.083333,0.025000,0.016667,0.100000,0.133333,0.0,1.000000,0.125,0.041667,0.000000,0.000000,0.041667,0.000000,1.000000
WAVEFORM,1.000000,0.975000,0.091667,0.758333,0.975000,0.958333,0.075000,0.083333,0.0,1.000000,0.125,0.900000,0.958333,0.975000,0.750000,1.000000,0.991667


# Gradual HT 100K - Percentage no labels: 96% - Metric: MDR

In [87]:
classifier = "trees.HoeffdingTree"
drift_size = 100000
#partialy_drift options: 95, 96, 97, 98, 99
partialy_drift = 96
type_drift = "Gradual"
#metric name options: detected changes, detected warnings, delay detection (average), delay true detection (average), MTFA (average), MDR, true changes detected, 
metric_name = "MDR"
dict_drift_type = {"gradual_agraw1": "AGRAW1", "gradual_agraw2": "AGRAW2", "gradual_led": "LED", 
                    "gradual_mixed": "MIXED", "gradual_randomRBF": "RANDOM_RBF", "gradual_sine": "SINE", "gradual_waveform": "WAVEFORM"}

df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        ##trees.HoeffdingTree_ADWIN_100000_95_0_AGRAW1_Abrupt
        files_path = glob.glob(f'../results/{drift_key}/{classifier}_{drift["id"]}_{drift_size}_{partialy_drift}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value, metric = metric_name)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,HypothesisTestDetectorKs,HypothesisTestDetectorTt,HypothesisTestDetectorWrs,MultivariateDetector,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2,Studd
generator,,,,,,,,,,,,,,,,,
AGRAW1,0.641667,0.491667,0.141667,0.266667,0.408333,0.141667,0.033333,0.050000,0.0,0.808333,0.125,0.158333,0.541667,0.416667,0.350000,0.625000,1.000000
AGRAW2,0.175000,0.058333,0.116667,0.091667,0.016667,0.016667,0.241667,0.258333,0.0,0.816667,0.125,0.008333,0.025000,0.008333,0.025000,0.050000,1.000000
LED,1.000000,1.000000,0.466667,1.000000,0.983333,0.833333,0.066667,0.075000,0.0,1.000000,0.125,1.000000,1.000000,0.108333,0.675000,0.166667,1.000000
MIXED,0.000000,0.000000,0.100000,0.025000,0.000000,0.000000,0.233333,0.191667,0.0,0.875000,0.125,0.016667,0.000000,0.000000,0.041667,0.008333,1.000000
RANDOM_RBF,0.000000,0.000000,0.116667,0.008333,0.000000,0.000000,0.075000,0.075000,0.0,1.000000,0.125,0.000000,0.000000,0.000000,0.000000,0.008333,0.741667
SINE,0.008333,0.016667,0.066667,0.058333,0.025000,0.008333,0.100000,0.100000,0.0,1.000000,0.125,0.008333,0.008333,0.008333,0.025000,0.008333,1.000000
WAVEFORM,1.000000,0.991667,0.125000,0.733333,0.991667,0.925000,0.158333,0.141667,0.0,1.000000,0.125,0.900000,0.958333,0.958333,0.858333,1.000000,1.000000


# Gradual HT 100K - Percentage no labels: 97% - Metric: MDR

In [88]:
classifier = "trees.HoeffdingTree"
drift_size = 100000
#partialy_drift options: 95, 96, 97, 98, 99
partialy_drift = 97
type_drift = "Gradual"
#metric name options: detected changes, detected warnings, delay detection (average), delay true detection (average), MTFA (average), MDR, true changes detected, 
metric_name = "MDR"
dict_drift_type = {"gradual_agraw1": "AGRAW1", "gradual_agraw2": "AGRAW2", "gradual_led": "LED", 
                    "gradual_mixed": "MIXED", "gradual_randomRBF": "RANDOM_RBF", "gradual_sine": "SINE", "gradual_waveform": "WAVEFORM"}

df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        ##trees.HoeffdingTree_ADWIN_100000_95_0_AGRAW1_Abrupt
        files_path = glob.glob(f'../results/{drift_key}/{classifier}_{drift["id"]}_{drift_size}_{partialy_drift}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value, metric = metric_name)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,HypothesisTestDetectorKs,HypothesisTestDetectorTt,HypothesisTestDetectorWrs,MultivariateDetector,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2,Studd
generator,,,,,,,,,,,,,,,,,
AGRAW1,0.691667,0.533333,0.200000,0.308333,0.533333,0.191667,0.066667,0.133333,0.0,0.891667,0.383333,0.325000,0.608333,0.558333,0.441667,0.658333,1.000000
AGRAW2,0.416667,0.233333,0.175000,0.175000,0.058333,0.025000,0.316667,0.316667,0.0,0.925000,0.383333,0.025000,0.141667,0.066667,0.025000,0.125000,1.000000
LED,1.000000,1.000000,0.583333,1.000000,1.000000,0.858333,0.100000,0.150000,0.0,1.000000,0.383333,1.000000,1.000000,0.041667,0.666667,0.266667,1.000000
MIXED,0.000000,0.000000,0.091667,0.041667,0.000000,0.000000,0.300000,0.200000,0.0,0.958333,0.383333,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
RANDOM_RBF,0.000000,0.000000,0.108333,0.016667,0.000000,0.000000,0.125000,0.125000,0.0,1.000000,0.383333,0.000000,0.000000,0.000000,0.000000,0.033333,0.808333
SINE,0.008333,0.000000,0.075000,0.058333,0.008333,0.008333,0.208333,0.150000,0.0,1.000000,0.383333,0.025000,0.000000,0.000000,0.041667,0.091667,1.000000
WAVEFORM,1.000000,0.975000,0.275000,0.716667,1.000000,0.975000,0.216667,0.275000,0.0,1.000000,0.383333,0.916667,0.983333,1.000000,0.858333,0.991667,1.000000


# Gradual HT 100K - Percentage no labels: 98% - Metric: MDR

In [89]:
classifier = "trees.HoeffdingTree"
drift_size = 100000
#partialy_drift options: 95, 96, 97, 98, 99
partialy_drift = 98
type_drift = "Gradual"
#metric name options: detected changes, detected warnings, delay detection (average), delay true detection (average), MTFA (average), MDR, true changes detected, 
metric_name = "MDR"
dict_drift_type = {"gradual_agraw1": "AGRAW1", "gradual_agraw2": "AGRAW2", "gradual_led": "LED", 
                    "gradual_mixed": "MIXED", "gradual_randomRBF": "RANDOM_RBF", "gradual_sine": "SINE", "gradual_waveform": "WAVEFORM"}

df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        ##trees.HoeffdingTree_ADWIN_100000_95_0_AGRAW1_Abrupt
        files_path = glob.glob(f'../results/{drift_key}/{classifier}_{drift["id"]}_{drift_size}_{partialy_drift}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value, metric = metric_name)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,HypothesisTestDetectorKs,HypothesisTestDetectorTt,HypothesisTestDetectorWrs,MultivariateDetector,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2,Studd
generator,,,,,,,,,,,,,,,,,
AGRAW1,0.725000,0.575000,0.250000,0.266667,0.641667,0.333333,0.175000,0.216667,0.0,0.858333,0.65,0.366667,0.625000,0.608333,0.575000,0.616667,1.00
AGRAW2,0.833333,0.283333,0.191667,0.158333,0.266667,0.100000,0.375000,0.283333,0.0,0.891667,0.65,0.083333,0.283333,0.316667,0.100000,0.433333,1.00
LED,1.000000,1.000000,0.791667,1.000000,0.983333,0.941667,0.300000,0.291667,0.0,1.000000,0.65,1.000000,1.000000,0.150000,0.558333,0.225000,1.00
MIXED,0.000000,0.000000,0.133333,0.041667,0.000000,0.000000,0.291667,0.191667,0.0,0.975000,0.65,0.008333,0.000000,0.000000,0.000000,0.100000,1.00
RANDOM_RBF,0.041667,0.000000,0.050000,0.008333,0.000000,0.000000,0.208333,0.225000,0.0,1.000000,0.65,0.000000,0.000000,0.033333,0.016667,0.200000,0.85
SINE,0.025000,0.000000,0.100000,0.058333,0.025000,0.016667,0.216667,0.191667,0.0,1.000000,0.65,0.016667,0.000000,0.050000,0.008333,0.075000,1.00
WAVEFORM,1.000000,0.991667,0.416667,0.691667,1.000000,0.991667,0.325000,0.333333,0.0,1.000000,0.65,0.958333,0.991667,0.975000,0.933333,0.983333,1.00


# Gradual HT 100K - Percentage no labels: 99% - Metric: MDR

In [90]:
classifier = "trees.HoeffdingTree"
drift_size = 100000
#partialy_drift options: 95, 96, 97, 98, 99
partialy_drift = 99
type_drift = "Gradual"
#metric name options: detected changes, detected warnings, delay detection (average), delay true detection (average), MTFA (average), MDR, true changes detected, 
metric_name = "MDR"
dict_drift_type = {"gradual_agraw1": "AGRAW1", "gradual_agraw2": "AGRAW2", "gradual_led": "LED", 
                    "gradual_mixed": "MIXED", "gradual_randomRBF": "RANDOM_RBF", "gradual_sine": "SINE", "gradual_waveform": "WAVEFORM"}

df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        ##trees.HoeffdingTree_ADWIN_100000_95_0_AGRAW1_Abrupt
        files_path = glob.glob(f'../results/{drift_key}/{classifier}_{drift["id"]}_{drift_size}_{partialy_drift}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value, metric = metric_name)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,HypothesisTestDetectorKs,HypothesisTestDetectorTt,HypothesisTestDetectorWrs,MultivariateDetector,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2,Studd
generator,,,,,,,,,,,,,,,,,
AGRAW1,0.808333,0.616667,0.541667,0.175000,0.666667,0.550000,0.416667,0.458333,0.000000,0.891667,0.883333,0.525000,0.675000,0.725000,0.616667,0.691667,1.000000
AGRAW2,0.975000,0.783333,0.375000,0.233333,0.725000,0.441667,0.433333,0.458333,0.008333,0.908333,0.883333,0.508333,0.850000,0.641667,0.391667,0.816667,1.000000
LED,1.000000,1.000000,0.891667,1.000000,1.000000,0.958333,0.516667,0.491667,0.000000,1.000000,0.883333,1.000000,1.000000,0.316667,0.458333,0.466667,1.000000
MIXED,0.025000,0.000000,0.025000,0.025000,0.000000,0.000000,0.233333,0.141667,0.000000,1.000000,0.883333,0.016667,0.000000,0.041667,0.000000,0.350000,1.000000
RANDOM_RBF,0.733333,0.000000,0.000000,0.025000,0.016667,0.000000,0.358333,0.275000,0.000000,1.000000,0.883333,0.000000,0.041667,0.425000,0.133333,0.525000,0.983333
SINE,0.191667,0.133333,0.100000,0.091667,0.116667,0.100000,0.300000,0.283333,0.000000,1.000000,0.883333,0.041667,0.150000,0.150000,0.091667,0.391667,1.000000
WAVEFORM,1.000000,0.991667,0.700000,0.716667,1.000000,1.000000,0.583333,0.541667,0.000000,1.000000,0.883333,0.975000,0.991667,0.991667,0.975000,1.000000,1.000000
